In [4]:
# from brat_parser import get_entities_relations_attributes_groups
import pandas as pd
import numpy as np
import csv
import json
from tqdm import tqdm

# from text_mining_func import normalize

# entities, relations, attributes, groups = get_entities_relations_attributes_groups("/Users/aelitta/Downloads/export/clinical/main/0/2.ann")

## Разметка RE

In [ ]:
import csv
import os

n = 0
ng = 0
nsg = 0
bio_annotation = []
entity = []
index = []
sentence = []

dir_ = '/Users/aelitta/Documents/DMMatrix/PAH/Marked_articles/'
#listfiles = os.listdir("Documents/Cancer/CLL/Разметка/")
listfiles = os.listdir(dir_)

prev = ''
jsons = []
jsons_neg = []

for file in listfiles:
    print(file)
    if file[-3:]=='tsv' and file[:3]!='cre':
        bio_annotation = []
        entity = []
        index = []
        sentence = []
        
        with open(dir_+file) as tsvfile:
            tsvreader = csv.reader(tsvfile, delimiter="\t")
            for line in tsvreader:
                print(line)
                if len(line)<5:
                    continue
                if line[5] !='_' and len(line[2])>0:
#                     print(line[2], line[6])
#                     if prev == '':
                    bio_annotation.append(line[2])
                    prev = line[4].split('[')[0]
                    index.append(line[6].split('[')[1])
                    entity.append(prev)
                    sentence.append(line[0])
#                     elif prev == line[6].split('[')[0]:
#                         bio_annotation.append(line[2])
#                         prev = line[6].split('[')[0]
#                         index.append(line[6].split('[')[1])



        for i in range(len(bio_annotation)):
            for j in range(len(bio_annotation)):
                if i!=j and i<len(entity) and j<len(entity) and entity[i]!=entity[j] and index[i]==index[j]:
                    print(i,j)
                    #извлекаем полную запись
                    k=0
                    while i+k< len(entity) and entity[i+k]==entity[i] and index[i+k]==index[i]:
                        k+=1
                    p=0
                    while j+p< len(entity) and  entity[j+p]==entity[j] and index[j+p]==index[j]:
                        p+=1
                    s=[]
                    with open(dir_+file) as tsvfile:
                        tsvreader = csv.reader(tsvfile, delimiter="\t")
                        for line in tsvreader:
                            if len(line)>0 and line[0].split('-')[0]==sentence[i].split('-')[0]:
                                s.append(line[2])
                    jsons.append({'h':[bio_annotation[i:i+k],entity[i], [[sentence[i+r].split('-')[1] for r in range(k)]]],
                                  't':[bio_annotation[j:j+p],entity[j], [[sentence[j+r].split('-')[1] for r in range(p)]]],
                                  'tokens':s
                                 })
                    i = i+k
                    j = j+p
                if i!=j and i<len(entity) and j<len(entity) and entity[i]!=entity[j] and index[i]!=index[j] \
                    and sentence[i].split('-')[0]==sentence[j].split('-')[0]:
                        #извлекаем полную запись
                        k=0
                        while i+k< len(entity) and entity[i+k]==entity[i] and index[i+k]==index[i]:
                            k+=1
                        p=0
                        while j+p< len(entity) and  entity[j+p]==entity[j] and index[j+p]==index[j]:
                            p+=1
                        s=[]
                        with open(dir_+file) as tsvfile:
                            tsvreader = csv.reader(tsvfile, delimiter="\t")
                            for line in tsvreader:
                                if len(line)>0 and line[0].split('-')[0]==sentence[i].split('-')[0]:
                                    s.append(line[2])
                            jsons_neg.append({'h':[bio_annotation[i:i+k],entity[i], [[sentence[i+r].split('-')[1] for r in range(k)]]],
                                          't':[bio_annotation[j:j+p],entity[j], [[sentence[j+r].split('-')[1] for r in range(p)]]],
                                          'tokens':s
                                         })
            
# f=open('Documents/Cancer/CLL/Разметка/TRAIN/train.tsv','w')
# cnt = 0
# for ele in bio_annotation:
# #     print(ele)
#     if cnt<=50:
#         f.write(ele+'\n')
#         cnt+=1
#     else:
#         f.write('\n')
#         cnt = 0
#         f.write(ele+'\n')

# f.close()

In [ ]:
import csv
import os

nfiles = 0
n = 0
ng = 0
nsg = 0
bio_annotation = []
entity = []
index = []
sentence = []
entity_unique = []

dir_ = '/Users/aelitta/Documents/DMMatrix/PAH/Marked_articles/'
#listfiles = os.listdir("Documents/Cancer/CLL/Разметка/")
listfiles = os.listdir(dir_)

prev = ''
jsons = []
jsons_neg_train = []
ent_ids = []

for file in listfiles:
    print(file)
    if file[-3:]=='tsv' and file[:3]!='cre' and file.find('ann_382')<0:
        nfiles+=1
        bio_annotation = []
        entity = []
        index = []
        sentence = []
        nconn = []
        ent_ids = []
        ent_dict = []
        dk=0
        with open(dir_+file) as tsvfile:
            tsvreader = csv.reader(tsvfile, delimiter="\t")
            for line in tsvreader:
                if len(line)<5:
                    continue
                if line[3]!= '_':
                    ent_dict.append([line[3],line[4]])
        df = pd.DataFrame(ent_dict)
        df1 = df[df[0]=='*'].reset_index().reset_index()
        nzs = df1.shape[0]+2
        df2 = df[df[0]!='*'].drop_duplicates().reset_index().reset_index()
        df2['level_0'] = df2['level_0']+df1.shape[0]+1
        dict_df2={}
        for t, b in zip(df2[1], df2['level_0']):
            dict_df2[t] = b
        nz = 0
        with open(dir_+file) as tsvfile:
            tsvreader = csv.reader(tsvfile, delimiter="\t")
            for line in tsvreader:
#                 print(line)
                if len(line)<5:
                    continue
                if line[5] !='_' and len(line[2])>0:
#                     print(line[2], line[6])
#                     if prev == '':
                    nc = len(line[6].split('|'))                      
                    prev = line[4].split('[')[0]
                    if prev not in entity_unique:
                        entity_unique.append(prev)
                    if line[4].find(']')>0:
                        ent_id = dict_df2[line[4]]
                    else:
                        ent_id = nz
                        nz+=1
#                     if ent_id>0:
#                         print(ent_id)
                    bio_annotation.append(line[2])
                    index.append([line[6].split('|')[r].split('[')[1].split(']')[0] for r in range(nc)])
                    entity.append(prev)
                    sentence.append(line[0])
                    ent_ids.append(ent_id)
                elif line[4] != '_' and len(line[2])>0:
                    prev = line[4].split('[')[0]
                    if prev not in entity_unique:
                        entity_unique.append(prev)
                    if line[4].find(']')>0:
                        ent_id = dict_df2[line[4]]
                    else:
                        ent_id = nz
                        nz+=1
                    if nz>nzs+1:
                        print('nz problem:',file, nz, nzs)
                    bio_annotation.append(line[2])
                    index.append([''])
                    entity.append(prev)
                    sentence.append(line[0])
                    ent_ids.append(ent_id)
#                     elif prev == line[6].split('[')[0]:
#                         bio_annotation.append(line[2])
#                         prev = line[6].split('[')[0]
#                         index.append(line[6].split('[')[1])


        i = 0
        k=1
        p=1
        while i<len(bio_annotation):
            k=1
            while i+k< len(entity) and entity[i+k]==entity[i] \
                and abs(int(sentence[i+k].split('-')[1])-int(sentence[i+(k-1)].split('-')[1]))==1:
                k+=1 ##весь токен
            j = i+k
            while j<len(bio_annotation):
#                 print(i,j)
                if i!=j and i<len(entity) and j<len(entity) and entity[i]!=entity[j] \
                    and sentence[i].split('-')[0]==sentence[j].split('-')[0]: 
                        #извлекаем полную запись
                        p=1
                        while j+p< len(entity) and  entity[j+p]==entity[j] \
                            and abs(int(sentence[j+p].split('-')[1])-int(sentence[j+p-1].split('-')[1]))==1:
                            p+=1
                        s=[]
                        with open(dir_+file) as tsvfile:
                            tsvreader = csv.reader(tsvfile, delimiter="\t")
                            for line in tsvreader:
                                if len(line)>0 and line[0].split('-')[0]==sentence[i].split('-')[0]:
                                    s.append(line[2])
                            ni = 0 #flag of conn
                            for nii in index[i]:
                                for nij in index[j]:
                                    if nii == nij and nii !='' and entity[i]!='_' and entity[j]!='_':
                                        jsons.append({'h':[bio_annotation[i:i+k],entity[i], [[sentence[i+r].split('-')[1] for r in range(k)]]],
                                                      't':[bio_annotation[j:j+p],entity[j], [[sentence[j+r].split('-')[1] for r in range(p)]]],
                                                      'tokens':s,
                                                      'h_id': ent_ids[i],
                                                      't_id': ent_ids[j],
                                                      'filename': file
                                                     })
                                        ni+=1
                            if ni==0 and entity[i]!='_' and entity[j]!='_':
                                jsons_neg_train.append({'h':[bio_annotation[i:i+k],entity[i], [[sentence[i+r].split('-')[1] for r in range(k)]]],
                                          't':[bio_annotation[j:j+p],entity[j], [[sentence[j+r].split('-')[1] for r in range(p)]]],
                                          'tokens':s,
                                          'h_id': ent_ids[i],
                                          't_id': ent_ids[j],
                                          'filename': file
                                                       })
                        j = j+p
                else:
                    j+=1
            i = i+k    


#         for i in range(len(bio_annotation)):
#             for j in range(len(bio_annotation)):
#                 if i!=j and i<len(entity) and j<len(entity) and entity[i]!=entity[j] and index[i]==index[j]:
#         #             print(i,j)
#                     #извлекаем полную запись
#                     k=0
#                     while i+k< len(entity) and entity[i+k]==entity[i] and index[i+k]==index[i]:
#                         k+=1
#                     p=0
#                     while j+p< len(entity) and  entity[j+p]==entity[j] and index[j+p]==index[j]:
#                         p+=1
#                     s=[]
#                     with open(dir_+file) as tsvfile:
#                         tsvreader = csv.reader(tsvfile, delimiter="\t")
#                         for line in tsvreader:
#                             if len(line)>0 and line[0].split('-')[0]==sentence[i].split('-')[0]:
#                                 s.append(line[2])
#                     jsons.append({'h':[bio_annotation[i:i+k],entity[i], [[sentence[i+r].split('-')[1] for r in range(k)]]],
#                                   't':[bio_annotation[j:j+p],entity[j], [[sentence[j+r].split('-')[1] for r in range(p)]]],
#                                   'tokens':s
#                                  })
#                     i = i+k
#                     j = j+p
#                 if i!=j and i<len(entity) and j<len(entity) and entity[i]!=entity[j] and index[i]!=index[j] \
#                     and sentence[i].split('-')[0]==sentence[j].split('-')[0]:
#                         #извлекаем полную запись
#                         k=0
#                         while i+k< len(entity) and entity[i+k]==entity[i] and index[i+k]==index[i]:
#                             k+=1
#                         p=0
#                         while j+p< len(entity) and  entity[j+p]==entity[j] and index[j+p]==index[j]:
#                             p+=1
#                         s=[]
#                         with open(dir_+file) as tsvfile:
#                             tsvreader = csv.reader(tsvfile, delimiter="\t")
#                             for line in tsvreader:
#                                 if len(line)>0 and line[0].split('-')[0]==sentence[i].split('-')[0]:
#                                     s.append(line[2])
#                             jsons_neg_train.append({'h':[bio_annotation[i:i+k],entity[i], [[sentence[i+r].split('-')[1] for r in range(k)]]],
#                                           't':[bio_annotation[j:j+p],entity[j], [[sentence[j+r].split('-')[1] for r in range(p)]]],
#                                           'tokens':s
#                                          })
#очистка отрицательных примеров
jsons_neg_clear = []
for k in range(len(jsons_neg_train)):
    if jsons_neg_train[k] in jsons:
        continue
    else:
        jsons_neg_clear.append(jsons_neg_train[k])
jsons_neg_train = jsons_neg_clear

In [ ]:
#inference для RE
import os

nfiles = 0
n = 0
ng = 0
nsg = 0
bio_annotation = []
entity = []
index = []
sentence = []

dir_ = '/Users/aelitta/Documents/DMMatrix/PAH/DownloadToInception/'
#listfiles = os.listdir("Documents/Cancer/CLL/Разметка/")
listfiles = os.listdir(dir_)

prev = ''
# jsons = []
jsons_neg = []

for file in listfiles:
    print(file)
    if file[-3:]=='txt' :
        nfiles+=1
        bio_annotation = []
        entity = []
        index = []
        sentence = []
        ent_ids = []
        ent_dict = []
        dk=0
        with open(dir_+file) as tsvfile:
            tsvreader = csv.reader(tsvfile, delimiter="\t")
            for line in tsvreader:
                if len(line)<5:
                    continue
                if line[3]!= '_':
                    ent_dict.append([line[3],line[4]])
        df = pd.DataFrame(ent_dict)
        df1 = df[df[0]=='*'].reset_index().reset_index()
        nzs = df1.shape[0]+2
        df2 = df[df[0]!='*'].drop_duplicates().reset_index().reset_index()
        df2['level_0'] = df2['level_0']+df1.shape[0]+1
        dict_df2={}
        for t, b in zip(df2[1], df2['level_0']):
            dict_df2[t] = b
        nz = 0
        with open(dir_+file) as tsvfile:
            tsvreader = csv.reader(tsvfile, delimiter="\t")
            for line in tsvreader:
#                 print(line)
                if len(line)<5:
                    continue
                if line[4] !='_' and len(line[2])>0:
#                     print(line[2], line[6])
#                     if prev == '':
                    bio_annotation.append(line[2])
                    prev = line[4].split('[')[0]
                    if line[4].find(']')>0:
                        ent_id = dict_df2[line[4]]
                    else:
                        ent_id = nz
                        nz+=1
                    if nz>nzs+1:
                        print('nz problem:',file, nz, nzs)
                    entity.append(prev)
                    sentence.append(line[0])
                    ent_ids.append(ent_id)
#                     elif prev == line[6].split('[')[0]:
#                         bio_annotation.append(line[2])
#                         prev = line[6].split('[')[0]
#                         index.append(line[6].split('[')[1])


        i = 0
        k=1
        p=1
        while i<len(bio_annotation):
            k=1
            while i+k< len(entity) and entity[i+k]==entity[i] \
                and abs(int(sentence[i+k].split('-')[1])-int(sentence[i+k-1].split('-')[1]))==1:
                k+=1 ##весь токен
            j = i+k
            while j<len(bio_annotation):
#                 print(i,j)
                if i!=j and i<len(entity) and j<len(entity) and entity[i]!=entity[j] \
                    and sentence[i].split('-')[0]==sentence[j].split('-')[0]: 
                        #извлекаем полную запись
                        p=1
                        while j+p< len(entity) and  entity[j+p]==entity[j] \
                            and abs(int(sentence[j+p].split('-')[1])-int(sentence[j+p-1].split('-')[1]))==1:
                            p+=1
                        s=[]
                        with open(dir_+file) as tsvfile:
                            tsvreader = csv.reader(tsvfile, delimiter="\t")
                            for line in tsvreader:
                                if len(line)>0 and line[0].split('-')[0]==sentence[i].split('-')[0]:
                                    s.append(line[2])
                            jsons_neg.append({'h':[bio_annotation[i:i+k],entity[i], [[sentence[i+r].split('-')[1] for r in range(k)]]],
                                          't':[bio_annotation[j:j+p],entity[j], [[sentence[j+r].split('-')[1] for r in range(p)]]],
                                          'tokens':s,
                                          'h_id': ent_ids[i],
                                          't_id': ent_ids[j]
                                         })                        
                        j = j+p
                else:
                    j+=1
            i = i+k    

In [8]:
#inference для RE с файлом
import os

nfiles = 0
n = 0
ng = 0
nsg = 0
bio_annotation = []
entity = []
index = []
sentence = []

# dir_ = '/Users/aelitta/Documents/DMMatrix/PAH/PAH_v2_project_2022-09-14_2106/annotation/'
dir_ = '/Users/aelitta/Documents/DMMatrix/PAH/готовые файлы tsv/'
#listfiles = os.listdir("Documents/Cancer/CLL/Разметка/")
listfiles = os.listdir(dir_)

prev = ''
# jsons = []
jsons_neg = []

for file in listfiles:
    f = file
    print(file)
#     file = file+'/sasha.tsv'
    if file[-3:]=='tsv' and file.find('Store')==-1 and f in dict_files.keys():
        nfiles+=1
        bio_annotation = []
        entity = []
        index = []
        sentence = []
        ent_ids = []
        ent_dict = []
        dk=0
        with open(dir_+file) as tsvfile:
            tsvreader = csv.reader(tsvfile, delimiter="\t")
            for line in tsvreader:
                if len(line)<5:
                    continue
                if line[3]!= '_':
                    ent_dict.append([line[3],line[4]])
        df = pd.DataFrame(ent_dict)
        if len(ent_dict)>0:
            df1 = df[df[0]=='*'].reset_index().reset_index()
            nzs = df1.shape[0]+2
            df2 = df[df[0]!='*'].drop_duplicates().reset_index().reset_index()
            df2['level_0'] = df2['level_0']+df1.shape[0]+1
            dict_df2={}
            for t, b in zip(df2[1], df2['level_0']):
                dict_df2[t] = b
            nz = 0
            with open(dir_+file) as tsvfile:
                tsvreader = csv.reader(tsvfile, delimiter="\t")
                for line in tsvreader:
    #                 print(line)
                    if len(line)<5:
                        continue
                    if line[4] !='_' and len(line[2])>0:
    #                     print(line[2], line[6])
    #                     if prev == '':

                        bio_annotation.append(line[2])
                        prev = line[4].split('[')[0]
                        if line[4].find(']')>0:
                            ent_id = dict_df2[line[4]]
                        else:
                            ent_id = nz
                            nz+=1
                        if nz>nzs+1:
                            print('nz problem:',file, nz, nzs)
                        entity.append(prev)
                        sentence.append(line[0])
                        ent_ids.append(ent_id)
    #                     elif prev == line[6].split('[')[0]:
    #                         bio_annotation.append(line[2])
    #                         prev = line[6].split('[')[0]
    #                         index.append(line[6].split('[')[1])


            i = 0
            k=1
            p=1
            while i<len(bio_annotation):
                k=1
                while i+k< len(entity) and entity[i+k]==entity[i] \
                    and abs(int(sentence[i+k].split('-')[1])-int(sentence[i+k-1].split('-')[1]))==1:
                    k+=1 ##весь токен
                j = i+k
                while j<len(bio_annotation):
    #                 print(i,j)
                    if i!=j and i<len(entity) and j<len(entity) and entity[i]!=entity[j] \
                        and sentence[i].split('-')[0]==sentence[j].split('-')[0]: 
                            #извлекаем полную запись
                            p=1
                            while j+p< len(entity) and  entity[j+p]==entity[j] \
                                and abs(int(sentence[j+p].split('-')[1])-int(sentence[j+p-1].split('-')[1]))==1:
                                p+=1
                            s=[]
                            with open(dir_+file) as tsvfile:
                                tsvreader = csv.reader(tsvfile, delimiter="\t")
                                for line in tsvreader:
                                    if len(line)>0 and line[0].split('-')[0]==sentence[i].split('-')[0]:
                                        s.append(line[2])
                                jsons_neg.append({'h':[bio_annotation[i:i+k],entity[i], [[sentence[i+r].split('-')[1] for r in range(k)]]],
                                              't':[bio_annotation[j:j+p],entity[j], [[sentence[j+r].split('-')[1] for r in range(p)]]],
                                              'tokens':s,
                                              'h_id': ent_ids[i],
                                              't_id': ent_ids[j],
                                              'filename': f.replace('.txt', '.tsv')
                                             })                        
                            j = j+p
                    else:
                        j+=1
                i = i+k  

ann_chest.13-1766.tsv
ann_j.ijcard.2017.02.094.tsv
ann_s00246-007-9139-2.tsv
ann_circulationaha.111.081125.tsv
ann_CIRCHEARTFAILURE.108.796789.tsv
ann_s10157-016-1344-y.tsv
ann_NEJMoa012212.tsv
ann_rccm.200308-1142OC.tsv
ann_s13318-017-0424-z.tsv
ann_113.full.tsv
ann_1343.full.tsv
ann_446.full.tsv
ann_s00392-005-0266-6.tsv
ann_11534940-000000000-00000.tsv
ann_j.pupt.2018.02.005.tsv
ann_j.healun.2012.04.005.tsv
ann_ajh_2Fhps057.tsv
ann_jcph.152.tsv
ann_rccm.201506-1091le.tsv
ann_bf02044111.tsv
ann_j.ijcard.2015.08.080.tsv
ann_pedsPAH.tsv
ann_chest.129.6.1636.tsv
ann_j.healun.2018.07.001.tsv
ann_j.pupt.2014.04.007.tsv
ann_ajrccm.163.4.2007116.tsv
ann_09031936.05.00075305.tsv
ann_11539110-000000000-00000.tsv
ann_200330.full.tsv
ann_s0100-879x2008000800003.tsv
ann_chest.14-0193.tsv
ann_1477-7525-11-31.tsv
ann_s00277-018-3518-z.tsv
ann_j.jacc.2006.12.037.tsv
ann_0091270011398241.tsv
ann_fjc.0b013e31827e0fa9.tsv
ann_heartjnl-2016-309621.tsv
ann_j.jpeds.2012.05.050.tsv
ann_journal.pone.011430

ann_j.ahj.2005.07.005.tsv
ann_j.1440-1754.1996.tb00942.x.tsv
ann_rccm.201507-1456OC.tsv
ann_art.34614.tsv
ann_asheducation-2011.1.419.tsv
ann_j.pupt.2005.09.006.tsv
ann_667.full.tsv
ann_154.full.tsv
ann_ham.2006.7.54.tsv
ann_cc7005.tsv
ann_eurheartj_2Fehu035.tsv
ann_1745-6215-14-188.tsv
ann_j.healun.2011.08.019.tsv
ann_ehm074.tsv
ann_1755-5922.12008.tsv
ann_ejhf.177.tsv
ann_s12325-013-0029-0.tsv
ann_j.chest.2017.04.188.tsv
ann_eurjhf_2Fhfs071.tsv
ann_clpt.2009.217.tsv
ann_asheducation-2017.1.423.tsv
ann_chest.07-0592.tsv
ann_j.jjcc.2014.01.003.tsv
ann_mtc.2003.107.tsv
ann_thx.2005.041954.tsv
ann_hr.2009.113.tsv
ann_498.full.tsv
ann_503.full.tsv
ann_j.healun.2006.09.016.tsv
ann_1465-9921-6-92.tsv
ann_s002469900463.tsv
ann_j.1365-2125.2009.03384.x.tsv
ann_biomolecules-10-01261-v3.tsv
ann_j.jtcvs.2003.11.035.tsv
ann_j.echo.2005.07.010.tsv
ann_pah-novinky-bwv.tsv
ann_09031936.00176312.tsv
ann_rccm.200410-1411OC.tsv
ann_rccm.201809-1631LE.tsv
ann_psp4.12202.tsv
ann_09031936.02.02572001.tsv


In [207]:
import os
dir_ = '/Users/aelitta/Documents/DMMatrix/PAH/Marked_articles/'
dir2 = '/Users/aelitta/Documents/DMMatrix/PAH/готовые файлы tsv'
# dir2 = '/Users/aelitta/Documents/DMMatrix/PAH/DownloadToInception/'
files = [f.replace('.txt','.tsv') for f in os.listdir(dir_)]+[f.replace('.txt','.tsv') for f in os.listdir(dir2)]
files = [f.replace('.tsv','.txt') for f in os.listdir(dir_)]+[f.replace('.tsv','.txt') for f in os.listdir(dir2)]
files = np.unique(dfresre['document'])


dict_files={}
for t, b in zip(np.unique(files), enumerate(np.unique(files))):
    dict_files[t] = b[0]+1

In [208]:
dict_files

{'00000539-200112000-00018.txt': 1,
 '00003088-200847090-00004.txt': 2,
 '000090555.txt': 3,
 '000107977.txt': 4,
 '000242498.txt': 5,
 '0003-4819-141-3-200408030-00005.txt': 6,
 '0003-4975_2895_2900408-d.txt': 7,
 '0003319711411704.txt': 8,
 '000355875.txt': 9,
 '000446583.txt': 10,
 '000450759.txt': 11,
 '00498254.2012.664665.txt': 12,
 '0091270004270833.txt': 13,
 '0091270007309708.txt': 14,
 '0091270008315315.txt': 15,
 '0091270008319793.txt': 16,
 '0091270009341182.txt': 17,
 '0091270009351173.txt': 18,
 '0091270011398241.txt': 19,
 '0091270011418656.txt': 20,
 '01.fjc.0000166207.74178.d0.txt': 21,
 '01.jcm.0000203850.97890.fe.txt': 22,
 '01.pdr.0000198772.26417.66.txt': 23,
 '0253-7613.132158.txt': 24,
 '030079905x30680.txt': 25,
 '030079905x46232.txt': 26,
 '030079906x167390.txt': 27,
 '03007990903210066.txt': 28,
 '03007995.2011.605440.txt': 29,
 '03007995.2012.685930.txt': 30,
 '09031936.00011308.txt': 31,
 '09031936.00105914.txt': 32,
 '09031936.00107012.txt': 33,
 '09031936.

In [ ]:
len(jsons) #[970]

In [ ]:
#для BERT+ linear classifier (ClinicalTransformerRE) - SEMVEAL

import json


path = 'Documents/DMMatrix/PAH/CRE/v4/'
train_size = int(len(jsons)/5)*4


train_types = []
with open(path+'train.tsv', 'w') as f:
#     f.write('index'+'\t'+'sentence'+'\t'+'label'+'\n')
    for i in range(train_size):
        ss1 = []
        ss2 = []
        h = ' '.join(jsons[i]['h'][0])
        t = ' '.join(jsons[i]['t'][0])
        h_id = str(dict_files[jsons[i]['filename']])+str(jsons[i]['h_id'])
        t_id = str(dict_files[jsons[i]['filename']])+str(jsons[i]['t_id'])
        k=0
        while k<len(jsons[i]['tokens']):
#             if jsons[i]['tokens'][k].find('.')==0:
#                 break
            if str(k+1) == jsons[i]['h'][2][0][0]:
                ss1.append('[s1]'+' '.join(jsons[i]['h'][0])+'[e1]')
                k = k+len(jsons[i]['h'][0])
            else:
                ss1.append(jsons[i]['tokens'][k])
                k+=1
        k=0
        while k <len(jsons[i]['tokens']):
#             if jsons[i]['tokens'][k].find('.')==0:
#                 break
            if str(k+1) == jsons[i]['t'][2][0][0]:
                ss2.append('[s2]'+' '.join(jsons[i]['t'][0])+'[e2]')
                k = k+len(jsons[i]['t'][0])
            else:
                ss2.append(jsons[i]['tokens'][k])
                k+=1
        type_ = min(jsons[i]['h'][1],jsons[i]['t'][1])+'_'+max(jsons[i]['h'][1],jsons[i]['t'][1])
        
#         if type_.lower().find('rate')>=0:
#             type_='cohort_effect'
#         elif type_.lower().find('duration')>=0:
#             type_='duration'
#         else:
#             type_ = 'cohort'
        if type_ not in train_types:
            train_types.append(type_)        
        line = type_ +'\t' + ' '.join(ss1) +'\t'+ ' '.join(ss2) +'\t'+ jsons[i]['h'][1] +'\t'+ jsons[i]['t'][1] +'\t'+\
        'T_'+h_id+'\t'+'T_'+t_id+str(i)+'\t'+'F_'+str(dict_files[jsons[i]['filename']])
#         print(i, line)
        if type_ in dev_types:
            f.write(line+'\n')
        
        ss1 = []
        ss2 = []
        h = ' '.join(jsons_neg_train[i]['h'][0])
        t = ' '.join(jsons_neg_train[i]['t'][0])
        h_id = str(dict_files[jsons_neg_train[i]['filename']])+str(jsons_neg_train[i]['h_id'])
        t_id = str(dict_files[jsons_neg_train[i]['filename']])+str(jsons_neg_train[i]['t_id'])
        k=0
        while k<len(jsons_neg_train[i]['tokens']):
#             if jsons_neg_train[i]['tokens'][k].find('.')==0:
#                 break
            if str(k+1) == jsons_neg_train[i]['h'][2][0][0]:
                ss1.append('[s1]'+' '.join(jsons_neg_train[i]['h'][0])+'[e1]')
                k = k+len(jsons_neg_train[i]['h'][0])
            else:
                ss1.append(jsons_neg_train[i]['tokens'][k])
                k+=1
        k=0
        while k <len(jsons_neg_train[i]['tokens']):
            if jsons_neg_train[i]['tokens'][k].find('.')==0:
                break
            if str(k+1) == jsons_neg_train[i]['t'][2][0][0]:
                ss2.append('[s2]'+' '.join(jsons_neg_train[i]['t'][0])+'[e2]')
                k = k+len(jsons_neg_train[i]['t'][0])
            else:
                ss2.append(jsons_neg_train[i]['tokens'][k])
                k+=1        

        if i%3==0:        
            line = 'NotRel' +'\t' + ' '.join(ss1) +'\t'+ ' '.join(ss2) +'\t'+ jsons_neg_train[i]['h'][1] +'\t'+ jsons_neg_train[i]['t'][1] +'\t'+\
            'T_'+h_id+'\t'+'T_'+t_id+'\t'+'F_'+str(dict_files[jsons_neg_train[i]['filename']])
    #         print(i, line)
            f.write(line+'\n')


f.close()
dev_types = []
with open(path+'dev.tsv', 'w') as f:
#     f.write('index'+'\t'+'sentence'+'\t'+'label'+'\n')
    for i in range(len(jsons)-train_size):
        if i<train_size:
            i = i+train_size
            ss1 = []
            ss2 = []
            h = ' '.join(jsons[i]['h'][0])
            t = ' '.join(jsons[i]['t'][0])
            h_id = str(dict_files[jsons[i]['filename']])+str(jsons[i]['h_id'])
            t_id = str(dict_files[jsons[i]['filename']])+str(jsons[i]['t_id'])
            k=0
            while k<len(jsons[i]['tokens']):
#                 if jsons[i]['tokens'][k].find('.')==0:
#                     break
                if str(k+1) == jsons[i]['h'][2][0][0]:
                    ss1.append('[s1]'+' '.join(jsons[i]['h'][0])+'[e1]')
                    k = k+len(jsons[i]['h'][0])
                else:
                    ss1.append(jsons[i]['tokens'][k])
                    k+=1
            k=0
            while k <len(jsons[i]['tokens']):
                if jsons[i]['tokens'][k].find('.')==0:
                    break
                if str(k+1) == jsons[i]['t'][2][0][0]:
                    ss2.append('[s2]'+' '.join(jsons[i]['t'][0])+'[e2]')
                    k = k+len(jsons[i]['t'][0])
                else:
                    ss2.append(jsons[i]['tokens'][k])
                    k+=1
            type_ = min(jsons[i]['h'][1],jsons[i]['t'][1])+'_'+max(jsons[i]['h'][1],jsons[i]['t'][1])
            
#             if type_.lower().find('rate')>=0:
#                 type_='cohort_effect'
#             elif type_.lower().find('duration')>=0:
#                 type_='duration'
#             else:
#                 type_ = 'cohort'

            line = type_ +'\t' + ' '.join(ss1) +'\t'+ ' '.join(ss2) +'\t'+ jsons[i]['h'][1] +'\t'+ jsons[i]['t'][1] +'\t'+\
            'T_'+h_id+'\t'+'T_'+t_id+'\t'+'F_'+str(dict_files[jsons[i]['filename']])
    #         print(i, line)
            if type_ in train_types:
                f.write(line+'\n')

            ss1 = []
            ss2 = []
            h = ' '.join(jsons_neg_train[i]['h'][0])
            t = ' '.join(jsons_neg_train[i]['t'][0])
            h_id = str(dict_files[jsons_neg_train[i]['filename']])+str(jsons_neg_train[i]['h_id'])
            t_id = str(dict_files[jsons_neg_train[i]['filename']])+str(jsons_neg_train[i]['t_id'])
            k=0
            while k<len(jsons_neg_train[i]['tokens']):
                if jsons_neg_train[i]['tokens'][k].find('.')==0:
                    break
                if str(k+1) == jsons_neg_train[i]['h'][2][0][0]:
                    ss1.append('[s1]'+' '.join(jsons_neg_train[i]['h'][0])+'[e1]')
                    k = k+len(jsons_neg_train[i]['h'][0])
                else:
                    ss1.append(jsons_neg_train[i]['tokens'][k])
                    k+=1
            k=0
            while k <len(jsons_neg_train[i]['tokens']):
                if jsons_neg_train[i]['tokens'][k].find('.')==0:
                    break
                if str(k+1) == jsons_neg_train[i]['t'][2][0][0]:
                    ss2.append('[s2]'+' '.join(jsons_neg_train[i]['t'][0])+'[e2]')
                    k = k+len(jsons_neg_train[i]['t'][0])
                else:
                    ss2.append(jsons_neg_train[i]['tokens'][k])
                    k+=1  
                    
            if i%3==0:
                line = 'NotRel' +'\t' + ' '.join(ss1) +'\t'+ ' '.join(ss2) +'\t'+ jsons_neg_train[i]['h'][1] +'\t'+ jsons_neg_train[i]['t'][1] +'\t'+\
                'T_'+h_id+'\t'+'T_'+t_id+'\t'+'F_'+str(dict_files[jsons_neg_train[i]['filename']])
        #         print(i, line)
                f.write(line+'\n')

            if type_ not in dev_types:
                dev_types.append(type_)
        
f.close()

In [ ]:
path = 'Documents/DMMatrix/PAH/CRE/v5/'
train_size = int(len(jsons)/5)*4

train_types = []
with open(path+'train.tsv', 'w') as f:
#     f.write('index'+'\t'+'sentence'+'\t'+'label'+'\n')
    for i in range(train_size):
        ss1 = []
        ss2 = []
        h = ' '.join(jsons[i]['h'][0])
        h_id = str(dict_files[jsons[i]['filename']])+str(jsons[i]['h_id'])
        t_id = str(dict_files[jsons[i]['filename']])+str(jsons[i]['t_id'])
        t = ' '.join(jsons[i]['t'][0])
        k=0
        while k<len(jsons[i]['tokens']):
#             if jsons[i]['tokens'][k].find('.')==0:
#                 break
            if str(k+1) == jsons_neg[i]['h'][2][0][0]:
                ss1.append('[s1]'+' '.join(jsons[i]['h'][0])+'[e1]')
                k = k+len(jsons[i]['h'][0])
            else:
                ss1.append(jsons[i]['tokens'][k])
                k+=1
        k=0
        while k <len(jsons[i]['tokens']):
#             if jsons[i]['tokens'][k].find('.')==0:
#                 break
            if str(k+1) == jsons[i]['t'][2][0][0]:
                ss2.append('[s2]'+' '.join(jsons[i]['t'][0])+'[e2]')
                k = k+len(jsons[i]['t'][0])
            else:
                ss2.append(jsons[i]['tokens'][k])
                k+=1
        type_ = min(jsons[i]['h'][1],jsons[i]['t'][1])+'_'+max(jsons[i]['h'][1],jsons[i]['t'][1])
        
#         if type_.lower().find('rate')>=0:
#             type_='cohort_effect'
#         elif type_.lower().find('duration')>=0:
#             type_='duration'
#         else:
#             type_ = 'cohort'
        if type_ not in train_types:
            train_types.append(type_)        
        line = 'Rel' +'\t' + ' '.join(ss1) +'\t'+ ' '.join(ss2) +'\t'+ jsons[i]['h'][1] +'\t'+ jsons[i]['t'][1] +'\t'+\
        'T_'+h_id+'\t'+'T_'+t_id+str(i)+'\t'+'F_'+str(dict_files[jsons[i]['filename']])
#         print(i, line)
        if type_ in train_types:
            f.write(line+'\n')
        
        ss1 = []
        ss2 = []
        h = ' '.join(jsons[i]['h'][0])
        h_id = str(dict_files[jsons_neg_train[i]['filename']])+str(jsons_neg_train[i]['h_id'])
        t_id = str(dict_files[jsons_neg_train[i]['filename']])+str(jsons_neg_train[i]['t_id'])
        t = ' '.join(jsons[i]['t'][0])
        k=0
        while k<len(jsons_neg_train[i]['tokens']):
#             if jsons_neg_train[i]['tokens'][k].find('.')==0:
#                 break
            if str(k+1) == jsons_neg_train[i]['h'][2][0][0]:
                ss1.append('[s1]'+' '.join(jsons_neg_train[i]['h'][0])+'[e1]')
                k = k+len(jsons_neg_train[i]['h'][0])
            else:
                ss1.append(jsons_neg_train[i]['tokens'][k])
                k+=1
        k=0
        while k <len(jsons_neg_train[i]['tokens']):
            if jsons_neg_train[i]['tokens'][k].find('.')==0:
                break
            if str(k+1) == jsons_neg_train[i]['t'][2][0][0]:
                ss2.append('[s2]'+' '.join(jsons_neg_train[i]['t'][0])+'[e2]')
                k = k+len(jsons_neg_train[i]['t'][0])
            else:
                ss2.append(jsons_neg_train[i]['tokens'][k])
                k+=1        

        if i%3==0:        
            line = 'NotRel' +'\t' + ' '.join(ss1) +'\t'+ ' '.join(ss2) +'\t'+ jsons_neg_train[i]['h'][1] +'\t'+ jsons_neg_train[i]['t'][1] +'\t'+\
            'T_'+h_id+'\t'+'T_'+t_id+'\t'+'F_'+str(dict_files[jsons_neg_train[i]['filename']])
    #         print(i, line)
            f.write(line+'\n')


f.close()
dev_types = []
with open(path+'dev.tsv', 'w') as f:
#     f.write('index'+'\t'+'sentence'+'\t'+'label'+'\n')
    for i in range(len(jsons)-train_size):
        if i<train_size:
            i = i+train_size
            ss1 = []
            ss2 = []
            h = ' '.join(jsons[i]['h'][0])
            t = ' '.join(jsons[i]['t'][0])
            h_id = str(dict_files[jsons[i]['filename']])+str(jsons[i]['h_id'])
            t_id = str(dict_files[jsons[i]['filename']])+str(jsons[i]['t_id'])
            k=0
            while k<len(jsons[i]['tokens']):
#                 if jsons[i]['tokens'][k].find('.')==0:
#                     break
                if str(k+1) == jsons[i]['h'][2][0][0]:
                    ss1.append('[s1]'+' '.join(jsons[i]['h'][0])+'[e1]')
                    k = k+len(jsons[i]['h'][0])
                else:
                    ss1.append(jsons[i]['tokens'][k])
                    k+=1
            k=0
            while k <len(jsons[i]['tokens']):
                if jsons[i]['tokens'][k].find('.')==0:
                    break
                if str(k+1) == jsons[i]['t'][2][0][0]:
                    ss2.append('[s2]'+' '.join(jsons[i]['t'][0])+'[e2]')
                    k = k+len(jsons[i]['t'][0])
                else:
                    ss2.append(jsons[i]['tokens'][k])
                    k+=1
            type_ = min(jsons[i]['h'][1],jsons[i]['t'][1])+'_'+max(jsons[i]['h'][1],jsons[i]['t'][1])
            
#             if type_.lower().find('rate')>=0:
#                 type_='cohort_effect'
#             elif type_.lower().find('duration')>=0:
#                 type_='duration'
#             else:
#                 type_ = 'cohort'

            line = 'Rel' +'\t' + ' '.join(ss1) +'\t'+ ' '.join(ss2) +'\t'+ jsons[i]['h'][1] +'\t'+ jsons[i]['t'][1] +'\t'+\
            'T_'+h_id+'\t'+'T_'+t_id+'\t'+'F_'+str(dict_files[jsons[i]['filename']])
    #         print(i, line)
            if type_ in train_types:
                f.write(line+'\n')

            ss1 = []
            ss2 = []
            h = ' '.join(jsons[i]['h'][0])
            t = ' '.join(jsons[i]['t'][0])
            h_id = str(dict_files[jsons_neg_train[i]['filename']])+str(jsons_neg_train[i]['h_id'])
            t_id = str(dict_files[jsons_neg_train[i]['filename']])+str(jsons_neg_train[i]['t_id'])
            k=0
            while k<len(jsons_neg_train[i]['tokens']):
                if jsons_neg_train[i]['tokens'][k].find('.')==0:
                    break
                if str(k+1) == jsons_neg_train[i]['h'][2][0][0]:
                    ss1.append('[s1]'+' '.join(jsons_neg_train[i]['h'][0])+'[e1]')
                    k = k+len(jsons_neg_train[i]['h'][0])
                else:
                    ss1.append(jsons_neg_train[i]['tokens'][k])
                    k+=1
            k=0
            while k <len(jsons_neg_train[i]['tokens']):
                if jsons_neg_train[i]['tokens'][k].find('.')==0:
                    break
                if str(k+1) == jsons_neg_train[i]['t'][2][0][0]:
                    ss2.append('[s2]'+' '.join(jsons_neg_train[i]['t'][0])+'[e2]')
                    k = k+len(jsons_neg_train[i]['t'][0])
                else:
                    ss2.append(jsons_neg_train[i]['tokens'][k])
                    k+=1  
                    
            if i%3==0:
                line = 'NotRel' +'\t' + ' '.join(ss1) +'\t'+ ' '.join(ss2) +'\t'+ jsons_neg_train[i]['h'][1] +'\t'+ jsons_neg_train[i]['t'][1] +'\t'+\
                'T_'+h_id+'\t'+'T_'+t_id+'\t'+'F_'+str(dict_files[jsons_neg_train[i]['filename']])
        #         print(i, line)
                f.write(line+'\n')

            if type_ not in dev_types:
                dev_types.append(type_)
 
        
f.close()

In [189]:
path = 'Documents/DMMatrix/PAH/CRE/v6_fs/'
train_size = int(len(jsons)/5)*4

train_types = []
with open(path+'train_fs.tsv', 'w') as f:
#     f.write('index'+'\t'+'sentence'+'\t'+'label'+'\n')
    for i in range(train_size):
        ss1 = []
        ss2 = []
        h = ' '.join(jsons[i]['h'][0])
        h_id = str(dict_files[jsons[i]['filename']])+str(jsons[i]['h_id'])
        t_id = str(dict_files[jsons[i]['filename']])+str(jsons[i]['t_id'])
        t = ' '.join(jsons[i]['t'][0])
        k=0
        while k<len(jsons[i]['tokens']):
#             if jsons[i]['tokens'][k].find('.')==0:
#                 break
            if str(k+1) == jsons_neg[i]['h'][2][0][0]:
                ss1.append('[s1]'+' '.join(jsons[i]['h'][0])+'[e1]')
                k = k+len(jsons[i]['h'][0])
            elif str(k+1) == jsons[i]['t'][2][0][0]:
                ss1.append('[s2]'+' '.join(jsons[i]['t'][0])+'[e2]')
                k = k+len(jsons[i]['t'][0])
            else:
                ss1.append(jsons[i]['tokens'][k])
                k+=1
        type_ = min(jsons[i]['h'][1],jsons[i]['t'][1])+'_'+max(jsons[i]['h'][1],jsons[i]['t'][1])
        
#         if type_.lower().find('rate')>=0:
#             type_='cohort_effect'
#         elif type_.lower().find('duration')>=0:
#             type_='duration'
#         else:
#             type_ = 'cohort'
        if type_ not in train_types:
            train_types.append(type_)        
        line = 'Rel' +'\t' + ' '.join(ss1) +'\t'+ jsons[i]['h'][1] +'\t'+ jsons[i]['t'][1] +'\t'+\
        'T_'+h_id+'\t'+'T_'+t_id+str(i)+'\t'+'F_'+str(dict_files[jsons[i]['filename']])
#         print(i, line)
        if type_ in train_types:
            f.write(line+'\n')
        
        ss1 = []
        ss2 = []
        h = ' '.join(jsons[i]['h'][0])
        h_id = str(dict_files[jsons_neg_train[i]['filename']])+str(jsons_neg_train[i]['h_id'])
        t_id = str(dict_files[jsons_neg_train[i]['filename']])+str(jsons_neg_train[i]['t_id'])
        t = ' '.join(jsons[i]['t'][0])
        k=0
        while k<len(jsons_neg_train[i]['tokens']):
#             if jsons_neg_train[i]['tokens'][k].find('.')==0:
#                 break
            if str(k+1) == jsons_neg_train[i]['h'][2][0][0]:
                ss1.append('[s1]'+' '.join(jsons_neg_train[i]['h'][0])+'[e1]')
                k = k+len(jsons_neg_train[i]['h'][0])
            elif str(k+1) == jsons_neg_train[i]['t'][2][0][0]:
                ss1.append('[s2]'+' '.join(jsons_neg_train[i]['t'][0])+'[e2]')
                k = k+len(jsons_neg_train[i]['t'][0])
            else:
                ss1.append(jsons_neg_train[i]['tokens'][k])
                k+=1
        if i%3==0:        
            line = 'NotRel' +'\t' + ' '.join(ss1) +'\t'+ jsons_neg_train[i]['h'][1] +'\t'+ jsons_neg_train[i]['t'][1] +'\t'+\
            'T_'+h_id+'\t'+'T_'+t_id+'\t'+'F_'+str(dict_files[jsons_neg_train[i]['filename']])
    #         print(i, line)
            f.write(line+'\n')


f.close()
dev_types = []
with open(path+'dev_fs.tsv', 'w') as f:
#     f.write('index'+'\t'+'sentence'+'\t'+'label'+'\n')
    for i in range(len(jsons)-train_size):
        if i<train_size:
            i = i+train_size
            ss1 = []
            ss2 = []
            h = ' '.join(jsons[i]['h'][0])
            t = ' '.join(jsons[i]['t'][0])
            h_id = str(dict_files[jsons[i]['filename']])+str(jsons[i]['h_id'])
            t_id = str(dict_files[jsons[i]['filename']])+str(jsons[i]['t_id'])
            k=0
            while k<len(jsons[i]['tokens']):
#                 if jsons[i]['tokens'][k].find('.')==0:
#                     break
                if str(k+1) == jsons[i]['h'][2][0][0]:
                    ss1.append('[s1]'+' '.join(jsons[i]['h'][0])+'[e1]')
                    k = k+len(jsons[i]['h'][0])
                elif str(k+1) == jsons[i]['t'][2][0][0]:
                    ss1.append('[s2]'+' '.join(jsons[i]['t'][0])+'[e2]')
                    k = k+len(jsons[i]['t'][0])
                else:
                    ss1.append(jsons[i]['tokens'][k])
                    k+=1
            type_ = min(jsons[i]['h'][1],jsons[i]['t'][1])+'_'+max(jsons[i]['h'][1],jsons[i]['t'][1])
            
#             if type_.lower().find('rate')>=0:
#                 type_='cohort_effect'
#             elif type_.lower().find('duration')>=0:
#                 type_='duration'
#             else:
#                 type_ = 'cohort'

            line = 'Rel' +'\t' + ' '.join(ss1) +'\t'+ jsons[i]['h'][1] +'\t'+ jsons[i]['t'][1] +'\t'+\
            'T_'+h_id+'\t'+'T_'+t_id+'\t'+'F_'+str(dict_files[jsons[i]['filename']])
    #         print(i, line)
            if type_ in train_types:
                f.write(line+'\n')

            ss1 = []
            ss2 = []
            h = ' '.join(jsons[i]['h'][0])
            t = ' '.join(jsons[i]['t'][0])
            h_id = str(dict_files[jsons_neg_train[i]['filename']])+str(jsons_neg_train[i]['h_id'])
            t_id = str(dict_files[jsons_neg_train[i]['filename']])+str(jsons_neg_train[i]['t_id'])
            k=0
            while k<len(jsons_neg_train[i]['tokens']):
                if jsons_neg_train[i]['tokens'][k].find('.')==0:
                    break
                if str(k+1) == jsons_neg_train[i]['h'][2][0][0]:
                    ss1.append('[s1]'+' '.join(jsons_neg_train[i]['h'][0])+'[e1]')
                    k = k+len(jsons_neg_train[i]['h'][0])
                elif str(k+1) == jsons_neg_train[i]['t'][2][0][0]:
                    ss1.append('[s2]'+' '.join(jsons_neg_train[i]['t'][0])+'[e2]')
                    k = k+len(jsons_neg_train[i]['t'][0])
                else:
                    ss1.append(jsons_neg_train[i]['tokens'][k])
                    k+=1
                    
            if i%3==0:
                line = 'NotRel' +'\t' + ' '.join(ss1) +'\t'+ jsons_neg_train[i]['h'][1] +'\t'+ jsons_neg_train[i]['t'][1] +'\t'+\
                'T_'+h_id+'\t'+'T_'+t_id+'\t'+'F_'+str(dict_files[jsons_neg_train[i]['filename']])
        #         print(i, line)
                f.write(line+'\n')

            if type_ not in dev_types:
                dev_types.append(type_)
 
        
f.close()

In [43]:
train_types

['dosage_drug',
 'drug_duration',
 '6MWD_drug',
 'drug_reason',
 'FC_reason',
 '6MWD_dosage',
 'FC_drug',
 'death\\_rate_drug',
 'duration_ntproBPN',
 '6MWD_duration',
 'death\\_rate_duration',
 '6mwd_drug',
 'drug_hospitalization',
 'dosage_duration']

In [191]:
#результат модели NER
dfresre = pd.read_csv('Documents/DMMatrix/PAH/CRE/spert/Блендинг объединеные токены.csv', delimiter = '\t')
dfresre.head()

,Unnamed: 0,tokens,tags,start,end,seq_id,document
0,0,NaN,-1,-1,-1,-1,-1
1,1,selective,O,0,9,1,00000539-200112000-00018.txt
2,2,pulmonary vasodilation,O,10,32,1,00000539-200112000-00018.txt
3,3,is an advantageous therapeutic,O,33,63,1,00000539-200112000-00018.txt
4,4,strategy,O,64,72,1,00000539-200112000-00018.txt


In [192]:
dfresrecnt = dfresre[dfresre['tags']!='O'].groupby('seq_id').count().reset_index()
dfresrecnt.head()

,seq_id,Unnamed: 0,tokens,tags,start,end,document
0,-1,1,0,1,1,1,1
1,2,4,4,4,4,4,4
2,3,8,8,8,8,8,8
3,4,5,5,5,5,5,5
4,5,7,7,7,7,7,7


In [193]:
dfresre = dfresre[dfresre['seq_id'].isin(dfresrecnt[dfresrecnt['document']>1]['seq_id'])]
dfresre.shape

(211662, 7)

In [33]:
sent = ' '.join(dfresre[dfresre['seq_id']==69348]['tokens'])
json_neg_test = []
seq_id = 69348
# starts = dfresre[(dfresre['seq_id']==seq_id)&(dfresre['tags']!='O')]['start']
starts = []
strt = 0
tokens = list(dfresre[dfresre['seq_id']==69348]['tokens'])
tags = list(dfresre[dfresre['seq_id']==69348]['tags'])
for token in tokens:
    starts.append(sent[strt:].find(token)+strt)
    strt+=len(token)+1
dc = pd.concat([pd.Series(tokens), pd.Series(tags), pd.Series(starts)], axis = 1)
dc.columns = ['tokens','tags','start']
dc = dc[dc['tags']!='O']
starts = dc[dc['tags']!='O']['start']
for tkn1s in starts:
    df1 = dc[(dc['start']==tkn1s)].copy()
    token1 = list(df1['tokens'])[0]
    pos1 = sent[max(tkn1s-len(token1),0):tkn1s+len(token1)+10].find(token1)+max(tkn1s-len(token1),0)
    s1 = len(sent[:pos1].split())
    for tkn2s in starts:
        if tkn1s<tkn2s: ##только последующие
            df2 = dc[dc['start']==tkn2s].copy()
            token2 = list(df2['tokens'])[0]
            pos2 = sent[max(tkn2s-len(token2),0):tkn2s+len(token2)+10].find(token2)+max(tkn2s-len(token2),0)
            s2 = len(sent[:pos2].split())
            if tkn1s < tkn2s and list(df1['tokens'])[0]!=list(df2['tokens'])[0]:
                 json_neg_test.append({
                       'h': [token1.split(' '), list(df1['tags'])[0], [str(y) for y in range(s1,s1+len(token1.split(' ')))]],
                       't': [token2.split(' '), list(df2['tags'])[0], [str(y) for y in range(s2,s2+len(token2.split(' ')))]],
                        'tokens': sent,
                        'h_id': cnt+tkn1s,
                        't_id': cnt+tkn2s,
                        'filename':filename
                    })

In [132]:
df1 = dfresre[(dfresre['seq_id']==seq_id)&(dfresre['start']==88)].copy()
df2 = dfresre[(dfresre['seq_id']==seq_id)&(dfresre['start']==113)].copy()
token1 = list(df1['tokens'])[0]
token2 = list(df2['tokens'])[0]

In [137]:
json_neg_test

[{'h': [['sublingual'], 'Drug', ['13']],
  't': [['lyophylized', 'sublingual'], 'Drug', ['19', '20']],
  'tokens': 'the mean plasma concentration versus time curves of sild following administration of sd sublingual tablets ( s7 ) , lyophylized sublingual tablets ( f4 ) and the commercial oral tablets ( revatio ) to human volunteers are shown in fig .',
  'h_id': 19571,
  't_id': 19596,
  'filename': 'v10007-012-0027-9.txt'},
 {'h': [['sublingual'], 'Drug', ['13']],
  't': [['oral'], 'Route', ['28']],
  'tokens': 'the mean plasma concentration versus time curves of sild following administration of sd sublingual tablets ( s7 ) , lyophylized sublingual tablets ( f4 ) and the commercial oral tablets ( revatio ) to human volunteers are shown in fig .',
  'h_id': 19571,
  't_id': 19651,
  'filename': 'v10007-012-0027-9.txt'},
 {'h': [['lyophylized', 'sublingual'], 'Drug', ['19', '20']],
  't': [['oral'], 'Route', ['28']],
  'tokens': 'the mean plasma concentration versus time curves of sild 

In [21]:
##for SEMVEAL

from tqdm import tqdm

json_neg = []
cnt = 0
for filename in tqdm(dfresre['document'].unique()):
    for seq_id in dfresre.loc[dfresre['document']==filename,'seq_id'].unique():
        sent = ' '.join(dfresre[dfresre['seq_id']==seq_id]['tokens'])
        starts = dfresre[(dfresre['seq_id']==seq_id)&(dfresre['tags']!='O')]['start']
        for tkn1s in starts:
            df1 = dfresre[(dfresre['seq_id']==seq_id)&(dfresre['start']==tkn1s)].copy()
            token1 = list(df1['tokens'])[0]
            pos1 = sent[max(tkn1s-len(token1),0):tkn1s+len(token1)+10].find(token1)+max(tkn1s-len(token1),0)
            s1 = len(sent[:pos1].split())
            for tkn2s in starts:
                if tkn1s<tkn2s: ##только последующие
                    df2 = dfresre[(dfresre['seq_id']==seq_id)&(dfresre['start']==tkn2s)].copy()
                    token2 = list(df2['tokens'])[0]
                    pos2 = sent[max(tkn2s-len(token2),0):tkn2s+len(token2)+10].find(token2)+max(tkn2s-len(token2),0)
                    s2 = len(sent[:pos2].split())
                    if tkn1s < tkn2s and list(df1['tokens'])[0]!=list(df2['tokens'])[0]:
                         json_neg.append({
                               'h': [token1.split(' '), list(df1['tags'])[0], [str(y) for y in range(s1,s1+len(token1.split(' ')))]],
                               't': [token2.split(' '), list(df2['tags'])[0], [str(y) for y in range(s2,s2+len(token2.split(' ')))]],
                                'tokens': sent,
                                'h_id': cnt+tkn1s,
                                't_id': cnt+tkn2s,
                                'filename':filename
                            })
        cnt+=1

100%|██████████| 442/442 [10:40<00:00,  1.45s/it]


In [194]:
from tqdm import tqdm

json_neg = []
cnt = 0
for filename in tqdm(dfresre['document'].unique()):
    for seq_id in dfresre.loc[dfresre['document']==filename,'seq_id'].unique():
        starts = []
        strt = 0
        tokens = list(dfresre[dfresre['seq_id']==seq_id]['tokens'])
        sent = ' '.join(tokens)
        tags = list(dfresre[dfresre['seq_id']==seq_id]['tags'])
        for token in tokens:
            starts.append(sent[strt:].find(token)+strt)
            strt+=len(token)+1
        dc = pd.concat([pd.Series(tokens), pd.Series(tags), pd.Series(starts)], axis = 1)
        dc.columns = ['tokens','tags','start']
        dc = dc[dc['tags']!='O']
        starts = dc[dc['tags']!='O']['start']
        for tkn1s in starts:
            df1 = dc[(dc['start']==tkn1s)].copy()
            token1 = list(df1['tokens'])[0]
            pos1 = sent[max(tkn1s-len(token1),0):tkn1s+len(token1)+10].find(token1)+max(tkn1s-len(token1),0)
            s1 = len(sent[:pos1].split())
            for tkn2s in starts:
                if tkn1s<tkn2s: ##только последующие
                    df2 = dc[dc['start']==tkn2s].copy()
                    token2 = list(df2['tokens'])[0]
                    pos2 = sent[max(tkn2s-len(token2),0):tkn2s+len(token2)+10].find(token2)+max(tkn2s-len(token2),0)
                    s2 = len(sent[:pos2].split())
                    if tkn1s < tkn2s and list(df1['tokens'])[0]!=list(df2['tokens'])[0]:
                         json_neg.append({
                               'h': [token1.split(' '), list(df1['tags'])[0], [str(y) for y in range(s1,s1+len(token1.split(' ')))]],
                               't': [token2.split(' '), list(df2['tags'])[0], [str(y) for y in range(s2,s2+len(token2.split(' ')))]],
                                'tokens': sent,
                                'h_id': tkn1s,
                                't_id': tkn2s,
                                'filename':filename,
                                 'seq_id':seq_id
                            })

100%|██████████| 442/442 [05:11<00:00,  1.42it/s]


In [39]:
json_neg[0]

{'h': [['milrinone'], 'Drug', ['3']],
 't': [['inhaled'], 'Route', ['32']],
 'tokens': "we hypothesized that milrinone , an adenosine - 3 ' , 5 ' - cyclic monophosphate ( camp ) - selective phosphodiesterase enzyme ( pde ) inhibitor may , when nebulized and inhaled , cause selective pulmonary vasodilation and potentiate the vasodilation by inhaled prostacyclin ( ipgi2 ) .",
 'h_id': 21,
 't_id': 170,
 'filename': '00000539-200112000-00018.txt'}

In [29]:
dfbio = pd.read_csv('/Users/aelitta/Documents/DMMatrix/PAH/CRE/spert/pred_entities_joint.csv')
dfbio.tag_name.value_counts()

reason              21781
drug                20927
duration            14571
dosage               5926
6mwd                 3346
prev_treat           2594
fc                   1880
nt-probnp             900
death_rate            670
hospitalization         6
progression_rate        2
Name: tag_name, dtype: int64

In [19]:
dict_files={}
for t, b in zip(np.unique(dfresre['document']), enumerate(np.unique(dfresre['document']))):
    dict_files[t] = b[0]+1

In [45]:
#для inference
path = 'Documents/DMMatrix/PAH/CRE/spert/'
from nltk import word_tokenize

jsons_neg = json_neg

with open(path+'test2.tsv', 'w') as f:
#     f.write('index'+'\t'+'sentence'+'\t'+'label'+'\n')
    for i in tqdm(range(len(jsons_neg))):
        #i = i+299968
        ss1 = []
        ss2 = []
        h = ' '.join(jsons_neg[i]['h'][0])
        t = ' '.join(jsons_neg[i]['t'][0])
        h_id = str(dict_files[jsons_neg[i]['filename']])+str(jsons_neg[i]['h_id'])
        t_id = str(dict_files[jsons_neg[i]['filename']])+str(jsons_neg[i]['t_id'])
        str_tokens = jsons_neg[i]['tokens'].split(' ')
#         str_tokens = word_tokenize(jsons_neg[i]['tokens'])
        k=0
        while k<len(str_tokens):
#             if jsons_neg[i]['tokens'][k].find('.')==0:
#                 break
            if str(k) == jsons_neg[i]['h'][2][0]:
                ss1.append('[s1]'+' '.join(jsons_neg[i]['h'][0])+'[e1]')
                k = k+len(jsons_neg[i]['h'][0])
#                 print(k)
            elif str(k) in jsons_neg[i]['h'][2]:
#                 print(k)
                continue
            else:
                ss1.append(str_tokens[k])
#                 print(k)
                k+=1
        k=0
        while k <len(str_tokens):
#             if jsons_neg[i]['tokens'][k].find('.')==0:
#                 break
            if str(k) == jsons_neg[i]['t'][2][0]:
                ss2.append('[s2]'+' '.join(jsons_neg[i]['t'][0])+'[e2]')
                k = k+len(jsons_neg[i]['t'][0])
            elif str(k) in jsons_neg[i]['t'][2]:
#                 print(k)
                continue
            else:
                ss2.append(str_tokens[k])
                k+=1
        
        line = 'NotRel' +'\t' + ' '.join(ss1) +'\t'+ ' '.join(ss2) +'\t'+ jsons_neg[i]['h'][1] +'\t'+ jsons_neg[i]['t'][1] +'\t'+\
        'T_'+h_id+'\t'+'T_'+t_id+'\t'+'F_'+str(dict_files[jsons_neg[i]['filename']])
#         print(line)
        f.write(line+'\n')   
        
f.close()

In [218]:
#для inference
path = 'Documents/DMMatrix/PAH/CRE/v6_fs/'
from nltk import word_tokenize

jsons_neg = json_neg

with open(path+'test_fs.tsv', 'w') as f:
#     f.write('index'+'\t'+'sentence'+'\t'+'label'+'\n')
    for i in tqdm(range(len(jsons_neg))):
        #i = i+299968
        ss1 = []
        ss2 = []
        h = ' '.join(jsons_neg[i]['h'][0])
        t = ' '.join(jsons_neg[i]['t'][0])
        h_id = str(dict_files[jsons_neg[i]['filename']])+str(jsons_neg[i]['h_id'])
        t_id = str(dict_files[jsons_neg[i]['filename']])+str(jsons_neg[i]['t_id'])
        str_tokens = jsons_neg[i]['tokens'].split(' ')
#         str_tokens = word_tokenize(jsons_neg[i]['tokens'])
        k=0
        while k<len(str_tokens):
#                 if jsons[i]['tokens'][k].find('.')==0:
#                     break
            if str(k+1) == jsons_neg[i]['h'][2][0][0]:
                ss1.append('[s1] '+' '.join(jsons_neg[i]['h'][0])+' [e1]')
                k = k+len(jsons_neg[i]['h'][0])
            elif str(k+1) == jsons_neg[i]['t'][2][0][0]:
                ss1.append('[s2] '+' '.join(jsons_neg[i]['t'][0])+' [e2]')
                k = k+len(jsons_neg[i]['t'][0])
            else:
                ss1.append(str_tokens[k])
                k+=1
        type_ = min(jsons_neg[i]['h'][1],jsons_neg[i]['t'][1])+'_'+max(jsons_neg[i]['h'][1],jsons_neg[i]['t'][1])

#             if type_.lower().find('rate')>=0:
#                 type_='cohort_effect'
#             elif type_.lower().find('duration')>=0:
#                 type_='duration'
#             else:
#                 type_ = 'cohort'



        
        line = 'NotRel' +'\t' + ' '.join(ss1) +'\t'+ jsons_neg[i]['h'][1] +'\t'+ jsons_neg[i]['t'][1] +'\t'+\
        'T_'+h_id+'\t'+'T_'+t_id+'\t'+'F_'+str(dict_files[jsons_neg[i]['filename']])
#         print(line)
        f.write(line+'\n')   
        
f.close()

100%|██████████| 149986/149986 [00:16<00:00, 9262.32it/s] 


In [217]:
test_fs = pd.read_csv(path+'test_fs.tsv', delimiter = '\t', header = None)

test_fs.head()

,0,1,2,3,4,5,6
0,NotRel,"we hypothesized [s1]milrinone[e1] milrinone , ...",Drug,Route,T_121,T_1170,F_1
1,NotRel,we hypothesized [s1]milrinone[e1] [s2]inhaled[...,Drug,Route,T_121,T_1254,F_1
2,NotRel,we hypothesized [s1]milrinone[e1] [s2]prostacy...,Drug,Drug,T_121,T_1262,F_1
3,NotRel,we hypothesized [s1]inhaled[e1] [s2]prostacycl...,Route,Drug,T_1170,T_1262,F_1
4,NotRel,"we hypothesized that [s1]inhaled[e1] , an aden...",Route,Drug,T_1254,T_1262,F_1


In [50]:
#for SEMVEAL 2
path = 'Documents/DMMatrix/PAH/CRE/spert/'
from nltk import word_tokenize

jsons_neg = json_neg

with open(path+'test2_with_name.tsv', 'w') as f:
#     f.write('index'+'\t'+'sentence'+'\t'+'label'+'\n')
    for i in tqdm(range(len(jsons_neg))):
#         i = i+22000
        ss1 = []
        ss2 = []
        h = ' '.join(jsons_neg[i]['h'][0])
        t = ' '.join(jsons_neg[i]['t'][0])
        h_id = str(dict_files[jsons_neg[i]['filename']])+str(jsons_neg[i]['h_id'])
        t_id = str(dict_files[jsons_neg[i]['filename']])+str(jsons_neg[i]['t_id'])
        str_tokens = jsons_neg[i]['tokens'].split(' ')
#         str_tokens = word_tokenize(jsons_neg[i]['tokens'])
        k=0
        while k<len(str_tokens):
#             if jsons_neg[i]['tokens'][k].find('.')==0:
#                 break
            if str(k) == jsons_neg[i]['h'][2][0]:
                ss1.append('[s1]'+' '.join(jsons_neg[i]['h'][0])+'[e1]')
                k = k+len(jsons_neg[i]['h'][0])
#                 print(k)
            elif str(k) in jsons_neg[i]['h'][2]:
#                 print(k)
                continue
            else:
                ss1.append(str_tokens[k])
#                 print(k)
                k+=1
        k=0
        while k <len(str_tokens):
#             if jsons_neg[i]['tokens'][k].find('.')==0:
#                 break
            if str(k) == jsons_neg[i]['t'][2][0]:
                ss2.append('[s2]'+' '.join(jsons_neg[i]['t'][0])+'[e2]')
                k = k+len(jsons_neg[i]['t'][0])
            elif str(k) in jsons_neg[i]['t'][2]:
#                 print(k)
                continue
            else:
                ss2.append(str_tokens[k])
                k+=1

        
        line = jsons_neg[i]['filename']+'\t'+' '.join(ss1) +'\t'+ ' '.join(ss2) +'\t'+ jsons_neg[i]['h'][1] +'\t'+ jsons_neg[i]['t'][1] +'\t'+\
        'T_'+h_id+'\t'+'T_'+t_id+'\t'+'F_'+str(dict_files[jsons_neg[i]['filename']]) +'\t'\
        +' '.join(jsons_neg[i]['h'][0])+'\t' +' '.join(jsons_neg[i]['t'][0])+'\t'+str(json_neg[i]['seq_id'])
#         print(line)
        f.write(line+'\n')   
        
f.close()

100%|██████████| 149986/149986 [00:23<00:00, 6307.89it/s]


In [ ]:
#для inference RE processing
path = 'Documents/DMMatrix/PAH/CRE/v5/'

with open(path+'test_with_name.tsv', 'w') as f:
#     f.write('index'+'\t'+'sentence'+'\t'+'label'+'\n')
    for i in range(len(jsons_neg)):
#         i = i+22000
        ss1 = []
        ss2 = []
        h = ' '.join(jsons_neg[i]['h'][0])
        t = ' '.join(jsons_neg[i]['t'][0])
        h_id = str(dict_files[jsons_neg[i]['filename']])+str(jsons_neg[i]['h_id'])
        t_id = str(dict_files[jsons_neg[i]['filename']])+str(jsons_neg[i]['t_id'])
        k=0
        while k<len(jsons_neg[i]['tokens']):
#             if jsons_neg[i]['tokens'][k].find('.')==0:
#                 break
            if str(k+1) == jsons_neg[i]['h'][2][0][0]:
                ss1.append('[s1]'+' '.join(jsons_neg[i]['h'][0])+'[e1]')
                k = k+len(jsons_neg[i]['h'][0])
            else:
                ss1.append(jsons_neg[i]['tokens'][k])
                k+=1
        k=0
        while k <len(jsons_neg[i]['tokens']):
#             if jsons_neg[i]['tokens'][k].find('.')==0:
#                 break
            if str(k+1) == jsons_neg[i]['t'][2][0][0]:
                ss2.append('[s2]'+' '.join(jsons_neg[i]['t'][0])+'[e2]')
                k = k+len(jsons_neg[i]['t'][0])
            else:
                ss2.append(jsons_neg[i]['tokens'][k])
                k+=1

        
        line = jsons_neg[i]['filename']+'\t'+' '.join(ss1) +'\t'+ ' '.join(ss2) +'\t'+ jsons_neg[i]['h'][1] +'\t'+ jsons_neg[i]['t'][1] +'\t'+\
        'T_'+h_id+'\t'+'T_'+t_id+str(i)+'\t'+'F_'+str(dict_files[jsons_neg[i]['filename']]) +'\t' +' '.join(jsons_neg[i]['h'][0])+'\t' +' '.join(jsons_neg[i]['t'][0])
#         print(line)
        f.write(line+'\n')
   
        
f.close()

In [ ]:
import pandas as pd
df = pd.read_csv(path+'test.tsv', delimiter = '\t',header=None)
df.shape

# Joint NER + RE by Santosh

In [181]:
import csv
import os
import re


pp = re.compile('(?<!\\\\)\'')

nfiles = 0
n = 0
ng = 0
nsg = 0
bio_annotation = []
entity = []
index = []
sentence = []
entity_unique = []

dir_ = '/Users/aelitta/Documents/DMMatrix/PAH/Marked_articles/'
# dir_ = '/Users/aelitta/Documents/DMMatrix/PAH/готовые файлы из инцепта/'
#listfiles = os.listdir("Documents/Cancer/CLL/Разметка/")
listfiles = os.listdir(dir_)

prev = ''
# jsons = [] #jsons for SEMVEAL
# jsons_neg_train = [] #jsons for SEMVEAL with no relation
ent_ids = [] #id of every token
entities = [] #entities for joint NER and RE model
relations = [] #relationss for joint NER and RE model
entities_unique = [] #id of complete entity
# jsonlist = [] #list of jsons for joint NER+RE model
# jsonlist_test = [] #list of jsons for joint NER+RE model, test
# sglobal = 0 #sentence counter

for file in listfiles:
    print(file)
    if file[-3:]=='tsv' and file[:3]!='cre' and file.find('ann_382')<0:
        nfiles+=1
        bio_annotation = [] #entities (tokens)
        entity = [] #type of entities
        index = [] #indexes for relations
        sentence = [] 
        nconn = []
        ent_ids = [] #id of every token
        ent_dict = []
        dk=0
        with open(dir_+file) as tsvfile:
            tsvreader = csv.reader(tsvfile, delimiter="\t")
            for line in tsvreader:
                if len(line)<5:
                    continue
                if line[3]!= '_':
                    ent_dict.append([line[3],line[4]])
        df = pd.DataFrame(ent_dict)
        df1 = df[df[0]=='*'].reset_index().reset_index()
        nzs = df1.shape[0]+2
        df2 = df[df[0]!='*'].drop_duplicates().reset_index().reset_index()
        df2['level_0'] = df2['level_0']+df1.shape[0]+1
        dict_df2={}
        for t, b in zip(df2[1], df2['level_0']):
            dict_df2[t] = b
        nz = 0
        with open(dir_+file) as tsvfile:
            tsvreader = csv.reader(tsvfile, delimiter="\t")
            for line in tsvreader:
#                 print(line)
                if len(line)<5:
                    continue
                if line[5] !='_' and len(line[2])>0:
#                     print(line[2], line[6])
#                     if prev == '':
                    nc = len(line[6].split('|'))                      
                    prev = line[4].split('[')[0]
                    if prev not in entity_unique:
                        entity_unique.append(prev)
                    if line[4].find(']')>0:
                        ent_id = dict_df2[line[4]]
                    else:
                        ent_id = nz
                        nz+=1
#                     if ent_id>0:
#                         print(ent_id)
                    bio_annotation.append(line[2])
                    index.append([line[6].split('|')[r].split('[')[1].split(']')[0] for r in range(nc)])
                    entity.append(prev)
                    sentence.append(line[0])
                    ent_ids.append(ent_id)
                elif line[4] != '_' and len(line[2])>0:
                    prev = line[4].split('[')[0]
                    if prev not in entity_unique:
                        entity_unique.append(prev)
                    if line[4].find(']')>0:
                        ent_id = dict_df2[line[4]]
                    else:
                        ent_id = nz
                        nz+=1
                    if nz>nzs+1:
                        print('nz problem:',file, nz, nzs)
                    bio_annotation.append(line[2])
                    index.append([''])
                    entity.append(prev)
                    sentence.append(line[0])
                    ent_ids.append(ent_id)
#                     elif prev == line[6].split('[')[0]:
#                         bio_annotation.append(line[2])
#                         prev = line[6].split('[')[0]
#                         index.append(line[6].split('[')[1])
        cnt_pat=0
        if os.path.exists('/Users/aelitta/Documents/DMMatrix/PAH/SPERT/PAH_v2_project_2022-10-31_1418/annotation/'+file[:-4]+'.txt/admin.tsv'):
            file1 = 'admin.tsv'
            cnt_pat =1
        elif os.path.exists('/Users/aelitta/Documents/DMMatrix/PAH/SPERT/PAH_v2_project_2022-10-31_1418/annotation/'+file[:-4]+'.txt/anna.tsv'):
            file1 = 'anna.tsv'
            cnt_pat =1
        if cnt_pat>0: 
            nfiles+=1
            bio_annotation = [] #entities (tokens)
            entity = [] #type of entities
            index = [] #indexes for relations
            sentence = [] 
            nconn = []
            ent_ids = [] #id of every token
            ent_dict = []
            dk=0
            with open('/Users/aelitta/Documents/DMMatrix/PAH/SPERT/PAH_v2_project_2022-10-31_1418/annotation/'+file[:-4]+'.txt/'+file1) as tsvfile:
                tsvreader = csv.reader(tsvfile, delimiter="\t")
                for line in tsvreader:
                    if len(line)<5:
                        continue
                    if line[3]!= '_':
                        ent_dict.append([line[3],line[4]])
            df = pd.DataFrame(ent_dict)
            df1 = df[df[0]=='*'].reset_index().reset_index()
            nzs = df1.shape[0]+2
            df2 = df[df[0]!='*'].drop_duplicates().reset_index().reset_index()
            df2['level_0'] = df2['level_0']+df1.shape[0]+1
            dict_df2={}
            for t, b in zip(df2[1], df2['level_0']):
                dict_df2[t] = b
            nz = 0


            with open('/Users/aelitta/Documents/DMMatrix/PAH/SPERT/PAH_v2_project_2022-10-31_1418/annotation/'+file[:-4]+'.txt/'+file1) as tsvfile:
                tsvreader = csv.reader(tsvfile, delimiter="\t")
                for line in tsvreader:
    #                 print(line)
                    if len(line)<5:
                        continue
                    if line[5] !='_' and len(line[2])>0 and line[5].find('cnt\_patients')>=0:
    #                     print(line[2], line[6])
    #                     if prev == '':
                        nc = len(line[6].split('|'))                      
                        prev = line[4].split('[')[0]
                        if prev not in entity_unique:
                            entity_unique.append(prev)
                        if line[4].find(']')>0:
                            ent_id = dict_df2[line[4]]
                        else:
                            ent_id = nz
                            nz+=1
    #                     if ent_id>0:
    #                         print(ent_id)
                        bio_annotation.append(line[2])
                        index.append([line[6].split('|')[r].split('[')[1].split(']')[0] for r in range(nc)])
                        entity.append(prev)
                        sentence.append(line[0])
                        ent_ids.append(ent_id)
                    elif line[4] != '_' and len(line[2])>0 and line[4].find('cnt\\_patients')>=0:
                        prev = line[4].split('[')[0]
                        if prev not in entity_unique:
                            entity_unique.append(prev)
                        if line[4].find(']')>0:
                            ent_id = dict_df2[line[4]]
                        else:
                            ent_id = nz
                            nz+=1
                        if nz>nzs+1:
                            print('nz problem:',file, nz, nzs)
                        bio_annotation.append(line[2])
                        index.append([''])
                        entity.append(prev)
                        sentence.append(line[0])
                        ent_ids.append(ent_id)




        i = 0
        k=1
        p=1
        scounter = 1 #sentences
        entities = []
        relations = []
        entities_unique = []
       
        while i<len(bio_annotation):
            if int(sentence[i].split('-')[0]) > scounter:
                if len(relations)>0 and sglobal%5!=0 and len(s)>0 and len(s)<80:
                    jsonlist.append({"tokens": s, "entities": entities, "relations": relations, "orig_id": sglobal})
                if len(relations)>0 and sglobal%5==0  and len(s)>0 and len(s)<80:
                        jsonlist_test.append({"tokens": s, "entities": entities, "relations": relations, "orig_id": sglobal})
                scounter=int(sentence[i].split('-')[0])
                sglobal+=1
                entities = []
                relations = []
                entities_unique = []
            k=1
            while i+k< len(entity) and entity[i+k]==entity[i] \
                and abs(int(sentence[i+k].split('-')[1])-int(sentence[i+(k-1)].split('-')[1]))==1:
                k+=1 ##весь токен
            if ent_ids[i] not in entities_unique and entity[i]!='*' and entity[i]!='_':
                entities_unique.append(ent_ids[i])
                entities.append({"type": entity[i].lower().replace("\\","").replace('ntprobpn','nt-probnp').\
                                 replace('bnp_rate','nt-probnp').replace('6mwd_rate', '6mwd').replace('functional_class','fc')\
                                 .replace('progression_rate','progression').replace('death','death_rate')\
                                 .replace('death_rate_rate','death_rate').replace('previous_treatment','prev_treat'), \
                                 "start": int(sentence[i].split('-')[1])-1, "end": int(sentence[i+k-1].split('-')[1])})

            j = i+k
            while j<len(bio_annotation):
#                 print(i,j)
                if i!=j and i<len(entity) and j<len(entity) and entity[i]!=entity[j] \
                    and sentence[i].split('-')[0]==sentence[j].split('-')[0]: 
                        #извлекаем полную запись
                        p=1
                        while j+p< len(entity) and  entity[j+p]==entity[j] \
                            and abs(int(sentence[j+p].split('-')[1])-int(sentence[j+p-1].split('-')[1]))==1:
                            p+=1
                        if ent_ids[j] not in entities_unique and entity[i]!='*' and entity[i]!='_':
                            entities_unique.append(ent_ids[j])
                            entities.append({"type": entity[j].lower().replace("\\","").replace('ntprobpn','nt-probnp').\
                                             replace('bnp_rate','nt-probnp').replace('6mwd_rate', '6mwd').replace('functional_class','fc').\
                                             replace('progression_rate','progression').replace('death_rate','death').\
                                             replace('death_rate_rate','death_rate').replace('previous_treatment','prev_treat'),\
                                             "start": int(sentence[j].split('-')[1])-1, "end": int(sentence[j+p-1].split('-')[1])})
                        s=[]
                        with open(dir_+file) as tsvfile:
                            tsvreader = csv.reader(tsvfile, delimiter="\t")
                            for line in tsvreader:
                                if len(line)>0 and line[0].split('-')[0]==sentence[i].split('-')[0]:
                                    if line[2].replace("'",'')!='':
                                        s.append(pp.sub('\"', str(line[2])))
                            ni = 0 #flag of conn
                            for nii in index[i]:
                                for nij in index[j]:
                                    if nii == nij and nii !='' and entity[i]!='_' and entity[j]!='_'  and entity[i]!='*' and entity[j]!='*':
                                        jsons.append({'h':[bio_annotation[i:i+k],entity[i], [[sentence[i+r].split('-')[1] for r in range(k)]]],
                                                      't':[bio_annotation[j:j+p],entity[j], [[sentence[j+r].split('-')[1] for r in range(p)]]],
                                                      'tokens':s,
                                                      'h_id': ent_ids[i],
                                                      't_id': ent_ids[j],
                                                      'filename': file
                                                     })
                                        relations.append({"type": "Rel", "head": entities_unique.index(ent_ids[i]), 
                                                          "tail":entities_unique.index(ent_ids[j])})
                                        ni+=1
                            if ni==0 and entity[i]!='_' and entity[j]!='_' and entity[i]!='*' and entity[j]!='*':
                                jsons_neg_train.append({'h':[bio_annotation[i:i+k],entity[i], [[sentence[i+r].split('-')[1] for r in range(k)]]],
                                          't':[bio_annotation[j:j+p],entity[j], [[sentence[j+r].split('-')[1] for r in range(p)]]],
                                          'tokens':s,
                                          'h_id': ent_ids[i],
                                          't_id': ent_ids[j],
                                          'filename': file
                                                       })
                                relations.append({"type": "NotRel", "head": entities_unique.index(ent_ids[i]), "tail":entities_unique.index(ent_ids[j])})
                        j = j+p
                else:
                    j+=1
            i = i+k    



ann_NEJMoa012212.tsv
ann_13993003.00364-2015.tsv
ann_13993003.01886-2017.tsv
ann_405.full.tsv
ann_NEJMoa1209655.tsv
ann_Phosphodisesterase inhibitors in Pulmonary hypertension.tsv
ann_09031936.04.00028404.tsv
ann_469.full.tsv
ann_CIRCULATIONAHA.104.473371.tsv
ann_NEJMoa1503184.tsv
ann_13993003.01030-2019.tsv
ann_000355875.tsv
ann_S2213-2600_2816_2930307-1.tsv
ann_1753465808103499.tsv
ann_S2213-2600_2816_2930019-4.tsv
ann_CIRCULATIONAHA.107.742510.tsv
ann_09031936.06.00030206.tsv
ann_13993003.02044-2014.tsv
ann_1745-6215-14-188.tsv
ann_1755-5922.12008.tsv
ann_S2213-2600_2814_2970013-X.tsv


In [182]:
opath = '/Users/aelitta/Documents/DMMatrix/PAH/CRE/spert/relations_marked_train_a.json'
with open(opath, "w") as ofile:
    json.dump(jsonlist, ofile) 
opath = '/Users/aelitta/Documents/DMMatrix/PAH/CRE/spert/relations_marked_test_a.json'
with open(opath, "w") as ofile:
    json.dump(jsonlist_test, ofile) 

In [183]:
import json
import scispacy
import spacy
from spacy.tokens import Doc
from more_itertools import locate


#!pip install scispacy
# !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_sm-0.2.4.tar.gz
#####################################
#### Customized tokenizer        ####
#####################################
nlp = spacy.load('en_core_web_sm')
#nlp = spacy.load("en_core_sci_sm")
# nlp = spacy.load('en_core_sci_sm')


def custom_tokenizer(text):
    tokens = text.split(" ")
    return Doc(nlp.vocab, tokens)
    #global tokens_dict
    #if text in tokens_dict:
    #   return Doc(nlp.vocab, tokens_dict[text])
    #else:
    #   VaueError("No tokenization for input text: ", text)

nlp.tokenizer = custom_tokenizer
#####################################


class JsonInputAugmenter():
    def __init__(self):
        basepath = '/Users/aelitta/Documents/DMMatrix/PAH/CRE/spert/'
        self.input_dataset_paths  = [basepath + 'relations_marked_train_a.json',
                                     basepath + 'relations_marked_test_a.json'
#                                       basepath + 'articles_inf_aa.json'
                                    ]
#         basepath = './data/datasets/ade/'
#         self.input_dataset_paths  = [basepath + 'ade_split_0_train.json', 
#                                      basepath + 'ade_split_0_test.json', 
#                                      basepath + 'ade_split_1_train.json',
#                                      basepath + 'ade_split_1_test.json',
#                                      basepath + 'ade_split_2_train.json',
#                                      basepath + 'ade_split_2_test.json',
#                                      basepath + 'ade_split_3_train.json',
#                                      basepath + 'ade_split_3_test.json',
#                                      basepath + 'ade_split_4_train.json',
#                                      basepath + 'ade_split_4_test.json',
#                                      basepath + 'ade_split_5_train.json',
#                                      basepath + 'ade_split_5_test.json',
#                                      basepath + 'ade_split_6_train.json',
#                                      basepath + 'ade_split_6_test.json',
#                                      basepath + 'ade_split_7_train.json',
#                                      basepath + 'ade_split_7_test.json',
#                                      basepath + 'ade_split_8_train.json',
#                                      basepath + 'ade_split_8_test.json',
#                                      basepath + 'ade_split_9_train.json',
#                                      basepath + 'ade_split_9_test.json'                                 
#                                      ]
#         self.output_dataset_paths = [basepath + 'ade_split_0_train_aug.json', 
#                                      basepath + 'ade_split_0_test_aug.json', 
#                                      basepath + 'ade_split_1_train_aug.json',
#                                      basepath + 'ade_split_1_test_aug.json',
#                                      basepath + 'ade_split_2_train_aug.json',
#                                      basepath + 'ade_split_2_test_aug.json',
#                                      basepath + 'ade_split_3_train_aug.json',
#                                      basepath + 'ade_split_3_test_aug.json',
#                                      basepath + 'ade_split_4_train_aug.json',
#                                      basepath + 'ade_split_4_test_aug.json',
#                                      basepath + 'ade_split_5_train_aug.json',
#                                      basepath + 'ade_split_5_test_aug.json',
#                                      basepath + 'ade_split_6_train_aug.json',
#                                      basepath + 'ade_split_6_test_aug.json',
#                                      basepath + 'ade_split_7_train_aug.json',
#                                      basepath + 'ade_split_7_test_aug.json',
#                                      basepath + 'ade_split_8_train_aug.json',
#                                      basepath + 'ade_split_8_test_aug.json',
#                                      basepath + 'ade_split_9_train_aug.json',
#                                      basepath + 'ade_split_9_test_aug.json'                                 
#                                      ]
        self.output_dataset_paths =  [basepath + 'relations_marked_train_aug.json',
                                     basepath + 'relations_marked_test_aug.json'
#                                       basepath + 'articles_inf_aa_aug.json'
                                     ]  
            
        #self.output_dataset_paths = [basepath + 'scierc_train_aug2.json', basepath + 'scierc_dev_aug2.json', basepath + 'scierc_train_dev_aug2.json', basepath + 'scierc_test_aug2.json']
        #self.taglist =[]

    def augment_docs_in_datasets(self):
        for ipath, opath  in zip(self.input_dataset_paths, self.output_dataset_paths):
            self._augment_docs(ipath, opath)
            #self._datasets[dataset_label] = dataset

    def _augment_docs(self, ipath, opath):
        global tokens_dict
        documents = json.load(open(ipath))
        augmented_documents = []
        nmultiroot=0
        for document in documents:
            jtokens = document['tokens']
            jrelations = document['relations']
            jentities = document['entities']
            jorig_id = document['orig_id']

            lower_jtokens = jtokens #[t.lower() for t in jtokens]
            text = ' '.join(lower_jtokens)
            #text = str.lower(text)
    
            #tokens_dict = {text: jtokens} #put the text in token_dict
            tokens = nlp(text)            #get annotated tokens
            jtags = [token.tag_ for token in tokens]
            #self.taglist =self.taglist + jtags
            jdeps = [token.dep_ for token in tokens]
            #"verb_indicator", "dep_head"
            #root = jdeps.index("ROOT") + 1 #as tokens are numbered from 1 by CoreNLP convention 
            vpos = list(locate(jdeps, lambda x: x == 'ROOT'))
            
            if (len(vpos) != 1):
                flag = 1
                nmultiroot += 1
                print("*** Full sentence:", text)
                for i in vpos:
                    print("ROOT [", i, "]: ", jtokens[i], ", pos tag: ", jtags[i], ", dep: ", jdeps[i])
            else:
                flag = 0

            verb_indicator = [0] * len(jdeps)
            for i in vpos:
                verb_indicator[i] = 1  

            jdep_heads = []
            for i, token in enumerate(tokens):
              if token.head == token:
                 token_idx = 0
              else:
                 token_idx = token.head.i - tokens[0].i + 1
              jdep_heads.append(token_idx)
            if (flag==1):
              print("dep_head: ", jdep_heads)
            d = {"tokens": jtokens, "pos_tags": jtags, "dep_label": jdeps, "verb_indicator": verb_indicator, "dep_head": jdep_heads, "entities": jentities, "relations": jrelations, "orig_id": jorig_id}
            augmented_documents.append(d)
        print("===============  #docs with multiroot = ", nmultiroot)
        with open(opath, "w") as ofile:
            json.dump(augmented_documents, ofile) 

In [49]:
nlp = spacy.load('en_core_web_sm')
# nlp.tokenizer.vocab.morphology.tag_map
# nlp = spacy.load("en_core_web_sm")
for label in nlp.get_pipe("parser").labels:
    print(label, " -- ", spacy.explain(label))

ROOT  --  None
acl  --  clausal modifier of noun (adjectival clause)
acomp  --  adjectival complement
advcl  --  adverbial clause modifier
advmod  --  adverbial modifier
agent  --  agent
amod  --  adjectival modifier
appos  --  appositional modifier
attr  --  attribute
aux  --  auxiliary
auxpass  --  auxiliary (passive)
case  --  case marking
cc  --  coordinating conjunction
ccomp  --  clausal complement
compound  --  compound
conj  --  conjunct
csubj  --  clausal subject
csubjpass  --  clausal subject (passive)
dative  --  dative
dep  --  unclassified dependent
det  --  determiner
dobj  --  direct object
expl  --  expletive
intj  --  interjection
mark  --  marker
meta  --  meta modifier
neg  --  negation modifier
nmod  --  modifier of nominal
npadvmod  --  noun phrase as adverbial modifier
nsubj  --  nominal subject
nsubjpass  --  nominal subject (passive)
nummod  --  numeric modifier
oprd  --  object predicate
parataxis  --  parataxis
pcomp  --  complement of preposition
pobj  --  ob

In [184]:
#augmentation from Santosh et. al
augmenter = JsonInputAugmenter()
augmenter.augment_docs_in_datasets()

*** Full sentence: Figure 2A demonstrates a signiﬁcant drop in the PA mean pressure for patients treated with bosentan ( baseline , 36 ± 7.1 mm Hg ; 16 weeks , 32 ± 8.8 mm Hg ; P , .02 ) .
ROOT [ 2 ]:  demonstrates , pos tag:  VBZ , dep:  ROOT
ROOT [ 32 ]:  Hg , pos tag:  NNP , dep:  ROOT
dep_head:  [2, 3, 0, 6, 6, 3, 6, 11, 11, 11, 7, 11, 12, 13, 14, 15, 16, 6, 18, 24, 24, 23, 24, 18, 6, 27, 6, 27, 32, 32, 32, 27, 0, 33, 33, 35, 35, 35, 33]
*** Full sentence: Figure 2B demonstrates no signiﬁcant change in the PA mean pressure for patients treated with placebo ( baseline , 30 ± 4.1 mm Hg ; 16 weeks , 31 ± 6.3 mm Hg ; P. .05 ) .
ROOT [ 2 ]:  demonstrates , pos tag:  VBZ , dep:  ROOT
ROOT [ 32 ]:  Hg , pos tag:  NNP , dep:  ROOT
dep_head:  [2, 3, 0, 6, 6, 3, 6, 11, 11, 11, 7, 11, 12, 13, 14, 15, 16, 16, 18, 21, 19, 23, 24, 21, 18, 27, 18, 27, 32, 32, 32, 27, 0, 33, 33, 35, 35, 33]
*** Full sentence: The fall in PVR in patients treated with bosentan ( 21.7 ± 2.75 Wood units ) was not sign

*** Full sentence: Daily warfarin dosages at week 18 were : 3.7 + 2.7 mg/ day for placebo patients , 2 .8 +1.2 mg/day for 50-mg sitaxsentan patients , 2 .1 + 1.0 mg/day for 100-mg sitaxsentan patients , and 5 . 1 + 2 .9 mg/day for OL bosentan patients .
ROOT [ 6 ]:  were , pos tag:  VBD , dep:  ROOT
ROOT [ 20 ]:  mg/day , pos tag:  NN , dep:  ROOT
dep_head:  [2, 3, 7, 3, 4, 5, 0, 7, 13, 9, 9, 13, 7, 13, 16, 14, 13, 13, 13, 21, 0, 21, 25, 25, 22, 25, 25, 25, 25, 31, 25, 25, 35, 35, 32, 25, 25, 39, 21, 39, 39, 39, 39, 39, 39, 48, 48, 45, 21]
*** Full sentence: Although the STRIDE-2 study was not powered to assess comparison between sitaxsentan 100 mg and OL bosentan , during this 18- week study , the incidence of elevated hepatic transaminases was 3 % for sitaxsentan 100 mg and 11 % for OL bosentan ( i.e. , the same as the rate reported in the bosentan package insert ) .
ROOT [ 30 ]:  was , pos tag:  VBD , dep:  ROOT
ROOT [ 47 ]:  same , pos tag:  JJ , dep:  ROOT
dep_head:  [7, 4, 4, 7, 

*** Full sentence: Results Of 376 patients receiving background endothelin receptor antagonist ( ERA ) and phosphodiesterase-5 inhibitor ( PDE-5i ) therapy , 115 had WHO FC II symptoms and 255 had WHO FC III symptoms at baseline .
ROOT [ 0 ]:  Results , pos tag:  NNS , dep:  ROOT
ROOT [ 21 ]:  had , pos tag:  VBD , dep:  ROOT
dep_head:  [0, 1, 4, 2, 4, 9, 8, 9, 5, 9, 9, 11, 9, 15, 9, 15, 18, 15, 22, 19, 19, 0, 26, 25, 26, 29, 26, 26, 22, 33, 32, 33, 29, 33, 34, 29]
*** Full sentence: Conclusion The addition of selexipag to background double combination therapy with an ERA and PDE-5i provides an incremental benefit similar to that seen in the overall population , including in patients with WHO FC II or III symptoms at baseline .
ROOT [ 0 ]:  Conclusion , pos tag:  NN , dep:  ROOT
ROOT [ 15 ]:  provides , pos tag:  VBZ , dep:  ROOT
dep_head:  [0, 3, 16, 3, 4, 7, 3, 10, 10, 7, 10, 13, 11, 13, 13, 0, 19, 19, 16, 19, 20, 21, 22, 23, 27, 27, 24, 27, 27, 29, 30, 31, 38, 35, 38, 35, 35, 32, 38

*** Full sentence: All patients received the following medications concomitantly after withdrawal of cardiopulmonary bypass : dopamine 5 μg/kg/min , olprinone 0.3 μg/kg/min and , if needed , dexmedetomidine 0.5–1 μg/kg/h .
ROOT [ 2 ]:  received , pos tag:  VBD , dep:  ROOT
ROOT [ 13 ]:  dopamine , pos tag:  NN , dep:  ROOT
ROOT [ 25 ]:  dexmedetomidine , pos tag:  NN , dep:  ROOT
dep_head:  [2, 3, 0, 6, 6, 3, 8, 3, 8, 9, 12, 10, 3, 0, 16, 14, 16, 26, 20, 18, 18, 18, 24, 18, 26, 0, 26, 26, 26]
*** Full sentence: The dose was then advanced every 3–4 days to 50– 100 mg three times daily , as tolerated , while we monitored symptoms ( dyspnea , chest pain , or lightheadedness with exertion , and lower extremity edema ) , systemic blood pressure , cardi- ac echocardiograms and 6 min walk distances .
ROOT [ 4 ]:  advanced , pos tag:  VBN , dep:  ROOT
ROOT [ 11 ]:  mg , pos tag:  NNP , dep:  ROOT
ROOT [ 42 ]:  pressure , pos tag:  NN , dep:  ROOT
dep_head:  [2, 3, 5, 5, 0, 8, 8, 5, 5, 9, 12, 0

*** Full sentence: The mean SMWT distance increased by 50 .94 m from 371 . 99 ± 78.73 m at baseline to 422 .94 ± 76.95 m at week 12 ( P < 0.0001 ) .
ROOT [ 3 ]:  distance , pos tag:  NN , dep:  ROOT
ROOT [ 15 ]:  m , pos tag:  NN , dep:  ROOT
ROOT [ 30 ]:  0.0001 , pos tag:  CD , dep:  ROOT
dep_head:  [4, 4, 4, 0, 4, 5, 9, 9, 6, 5, 10, 4, 16, 16, 16, 0, 16, 17, 16, 22, 22, 19, 24, 16, 24, 25, 26, 29, 31, 31, 0, 31, 31]
*** Full sentence: The mean pulmonary artery pressure decreased by 8 % ( 80.19 ± 20.76 mmHg at baseline and 73.77 ± 18.24 mmHg at week 12 ; P = 0.0002 ) .
ROOT [ 5 ]:  decreased , pos tag:  VBD , dep:  ROOT
ROOT [ 20 ]:  mmHg , pos tag:  CD , dep:  ROOT
ROOT [ 27 ]:  0.0002 , pos tag:  CD , dep:  ROOT
dep_head:  [5, 5, 5, 5, 6, 0, 6, 9, 7, 9, 12, 9, 14, 12, 12, 15, 12, 19, 12, 21, 0, 28, 22, 23, 28, 27, 28, 0, 28, 28]
*** Full sentence: The most frequent adverse events were headache and flushing , which were reported in 5.0 % and 1 . 67 % in our patients , respectively .

...

*** Full sentence: Iloprost was administered at an increas- ing dose to a target of 10 µg 4 – 6 times/ d ( Fig .
ROOT [ 2 ]:  administered , pos tag:  VBN , dep:  ROOT
ROOT [ 20 ]:  Fig , pos tag:  NNP , dep:  ROOT
dep_head:  [3, 3, 0, 3, 6, 4, 6, 7, 7, 11, 9, 11, 14, 12, 14, 14, 14, 19, 21, 19, 0, 21]
*** Full sentence: Outcome Measures The primary end point was changes in peak 6-min walk distance ( 6MWD ) that were defined as within 10 – 60 min after iloprost inhalation at week 6 and 3 months after the initiation of treatment compared with baseline values .
ROOT [ 1 ]:  Measures , pos tag:  VBZ , dep:  ROOT
ROOT [ 6 ]:  was , pos tag:  VBD , dep:  ROOT
dep_head:  [2, 0, 6, 6, 6, 7, 0, 7, 8, 13, 13, 13, 9, 13, 13, 13, 19, 19, 13, 19, 20, 21, 24, 25, 19, 19, 28, 26, 28, 29, 30, 28, 34, 35, 19, 37, 35, 37, 38, 37, 40, 43, 41, 7]
*** Full sentence: An increase of 133.75 ± 25.6 m was observed in subjects receiving combination therapy , 0.86 ± 17.6 m in value ( Table 2 ) .
ROOT [ 1 ]:  inc

*** Full sentence: Eligible patients also had a mean pulmonary artery pressure ( mPAP ) .30 mmHg , a pulmonary capillary wedge pressure ( PCWP ) ,15 mmHg and a pulmonary vascular resistance ( PVR ) .400 dyn ? s-1 ? cm-5 , as demonstrated by right heart catheterisa- tion within 3 months prior to the commencement of the study .
ROOT [ 3 ]:  had , pos tag:  VBD , dep:  ROOT
ROOT [ 13 ]:  mmHg , pos tag:  NNP , dep:  ROOT
ROOT [ 23 ]:  ,15 , pos tag:  NFP , dep:  ROOT
ROOT [ 24 ]:  mmHg , pos tag:  NNP , dep:  ROOT
ROOT [ 34 ]:  dyn , pos tag:  NNP , dep:  ROOT
ROOT [ 37 ]:  ? , pos tag:  . , dep:  ROOT
ROOT [ 38 ]:  cm-5 , pos tag:  NNP , dep:  ROOT
dep_head:  [2, 4, 4, 0, 9, 9, 9, 9, 4, 9, 9, 9, 9, 0, 14, 20, 20, 20, 20, 14, 20, 20, 20, 0, 0, 25, 30, 30, 30, 25, 32, 30, 32, 30, 0, 35, 38, 0, 0, 39, 42, 39, 42, 45, 47, 45, 43, 42, 50, 48, 50, 51, 54, 52, 54, 57, 55, 39]
*** Full sentence: During the long-term extension , a further six patients discontinued therapy permanently : one patien

*** Full sentence: After 16 weeks of therapy , PA mean pressure fell signiﬁcantly for the bosentan vs the placebo group ( 24 ± 6.6 mm Hg vs 1 ± 3.7 mm Hg , respectively ; P , .05 ) .
ROOT [ 9 ]:  fell , pos tag:  VBD , dep:  ROOT
ROOT [ 23 ]:  Hg , pos tag:  NNP , dep:  ROOT
dep_head:  [10, 3, 1, 3, 4, 10, 9, 9, 10, 0, 10, 10, 14, 12, 14, 18, 18, 15, 10, 23, 23, 21, 10, 0, 24, 27, 30, 29, 30, 25, 24, 24, 24, 24, 34, 34, 34, 24]
*** Full sentence: ORIGINAL ARTICLE Haemodynamic effects of riociguat in inoperable/ recurrent chronic thromboembolic pulmonary hypertension Methods Patients with inoperable or persistent/ recurrent CTEPH ( n=261 ; mean± SD age 59±14 years ; 66 % women ) were randomised to riociguat ( up to −213 ) ; p < 0.0001 ) and persistent/recurrent ( n=72 ; patients .
ROOT [ 3 ]:  effects , pos tag:  NNS , dep:  ROOT
ROOT [ 35 ]:  randomised , pos tag:  VBN , dep:  ROOT
dep_head:  [4, 3, 4, 0, 4, 5, 4, 4, 13, 13, 13, 13, 15, 15, 36, 15, 21, 17, 17, 17, 16, 23, 21, 23, 27, 2

*** Full sentence: In contrast , among the patients who received placebo , only 1 ( 5 % ) of 20 patients improved from class III to class II , 15 ( 75 % ) remained stable , two ( 10 % ) de- teriorated from class II to class III , and two deteriorated from class III to class IV ( the two patients who died were classified as class IV ) ( P 0.585 ) ( Figure 3 ) .
ROOT [ 19 ]:  improved , pos tag:  VBN , dep:  ROOT
ROOT [ 41 ]:  teriorated , pos tag:  VBN , dep:  ROOT
ROOT [ 71 ]:  P , pos tag:  NN , dep:  ROOT
ROOT [ 75 ]:  Figure , pos tag:  NN , dep:  ROOT
dep_head:  [20, 1, 20, 15, 6, 4, 8, 6, 8, 15, 12, 15, 15, 15, 20, 15, 15, 19, 17, 0, 20, 23, 21, 25, 20, 25, 25, 31, 31, 31, 25, 25, 20, 33, 33, 39, 39, 39, 33, 39, 42, 0, 42, 45, 43, 47, 42, 47, 42, 42, 52, 42, 52, 55, 53, 57, 52, 57, 58, 62, 62, 66, 64, 62, 66, 52, 66, 69, 67, 52, 72, 0, 72, 72, 76, 0, 76, 76, 76]
*** Full sentence: DISCUSSION The current study is the first randomized , placebo-controlled tri- al to evaluate the eff

*** Full sentence: Olschewski et al [ 9 ] described the use of aerosolized iloprost , a carbacyclin analogue of PGI2 , for severe pulmonary hypertension .
ROOT [ 0 ]:  Olschewski , pos tag:  NNP , dep:  ROOT
ROOT [ 2 ]:  al , pos tag:  NNP , dep:  ROOT
ROOT [ 6 ]:  described , pos tag:  VBD , dep:  ROOT
dep_head:  [0, 3, 0, 5, 7, 5, 0, 9, 7, 9, 12, 10, 12, 16, 16, 12, 16, 17, 12, 7, 23, 23, 20, 7]
*** Full sentence: 6MWT distance ( 52.53 m , 95 % CI =3.65 93.24 m , Cohen ’ s d = 0.45 moderate , power= 35 % ) and % 6MWT ( 6 . 73 % , 95 % CI =0.43 12 . 08 % ) sig- niﬁcantly and clinically ( > 33 m ) increased within TG after AET ( p < 0.05 ) .
ROOT [ 1 ]:  distance , pos tag:  NN , dep:  ROOT
ROOT [ 9 ]:  =3.65 , pos tag:  NFP , dep:  ROOT
ROOT [ 11 ]:  m , pos tag:  NN , dep:  ROOT
ROOT [ 16 ]:  d , pos tag:  NN , dep:  ROOT
ROOT [ 19 ]:  moderate , pos tag:  JJ , dep:  ROOT
ROOT [ 32 ]:  % , pos tag:  NN , dep:  ROOT
ROOT [ 38 ]:  12 , pos tag:  CD , dep:  ROOT
ROOT [ 41 ]:  % , pos ta

In [ ]:
# !pip install -U pip setuptools wheel
# !pip install -U spacy
# !python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_sci_sm')

In [ ]:
import en_core_sci_sm  

In [22]:
sent = pd.read_csv('/Users/aelitta/Documents/DMMatrix/PAH/CRE/spert/sentences.csv', delimiter = '\t')
sent.head()

,sentence_id,filename,sentence
0,1,00000539-200112000-00018.txt,Selective pulmonary vasodilation is an advanta...
1,2,00000539-200112000-00018.txt,"We hypothesized that milrinone, an adenosine-3..."
2,3,00000539-200112000-00018.txt,"Consequently, we investigated the hemodynamic ..."
3,4,00000539-200112000-00018.txt,During mechanical ventilation and using a conv...
4,5,00000539-200112000-00018.txt,"In the same manner, 11 patients received iPGI2..."


In [23]:
from nltk import word_tokenize
jsonlist_a = []
jsonlist_f = []
sglobal = 0
for i in range(sent.shape[0]):
    st = sent['sentence'][i]
    file = sent['filename'][i]
    s = []
    for t in word_tokenize(st):
        s.append(pp.sub('\"', t))
    if len(s)>5 and len(s)<60:
        entities = [{"type": "drug", "start":0, "end": 1}, {"type":"reason", "start":1, "end":2}]
        relations = [{"type":"NotRel", "head": 0, "tail":1}]
        jsonlist_a.append({"tokens": s, "entities": entities, "relations": relations, "orig_id": sglobal})
        jsonlist_f.append({"tokens": s, "entities": entities, "relations": relations, "orig_id": sglobal, "sentence": sent['sentence_id'][i], "file":file})
        sglobal+=1

In [206]:
with open('/Users/aelitta/Documents/DMMatrix/PAH/CRE/spert/articles_inf_a.json', 'w') as f:
    f.write(json.dumps(jsonlist_a))

In [175]:
#inference for my own model based on Roberta

pp = re.compile('(?<!\\\\)\'')

nfiles = 0
n = 0
ng = 0
nsg = 0
bio_annotation = []
entity = []
index = []
sentence = []
entity_unique = []

dir_ = '/Users/aelitta/Documents/DMMatrix/PAH/DownloadToInception/'
# dir_ = '/Users/aelitta/Documents/DMMatrix/PAH/готовые файлы tsv/'
#listfiles = os.listdir("Documents/Cancer/CLL/Разметка/")
listfiles = os.listdir(dir_)

entities = [] #entities for joint NER and RE model
relations = [] #relationss for joint NER and RE model

jsonlist_inf = [] #list of jsons for joint NER+RE model
sglobal = 0 #sentence counter

for file in listfiles:
    print(file)
    if file[-3:]=='txt' :
        nfiles+=1
        entities = []
        tokens = []
        relations = []
        token_counter = 0 
        ent_token_counter = 0
        sglobal = 0
        with open(dir_+file) as tsvfile:
            tsvreader = csv.reader(tsvfile, delimiter=" ")
            for line in tsvreader:
#                 print(line)
                if len(line) == 0:
                    relations = []
                    for h in range(len(entities)):
                        for t in range(len(entities)):
                            if h!=t:
                                relations.append({"type":"NotRel", "head": h, "tail":t})
                    jsonlist_inf.append({"tokens": tokens, "entities": entities, "relations": relations, "orig_id": sglobal})
                    tokens = []
                    relations = []
                    entities = []
                    token_counter = 0 
                    ent_token_counter = 0
                    sglobal+=1
                
                else:
                    if line[1].find('B-')==0:
                        ent_start = token_counter
                        ent_token_counter+=1
                        ent_type = line[1].split('-')[1].lower().replace('ntprobpn','nt-probnp').\
                        replace('progression_rate','progression').replace('hospitalization_rate','hospitalization').\
                        replace('death', 'death_rate').replace('death_rate_rate','death_rate')
                    if line[1].find('I-')==0 and ent_token_counter>0:
                        ent_token_counter+=1
                    if line[1] == 'O' and ent_token_counter >0:
                            ent_end = ent_start+ent_token_counter
                            ent_token_counter = 0 
                            entities.append({"type": ent_type, "start": ent_start, "end": ent_end})
                    token_counter+=1
                    tokens.append(line[0])
                     

ann_j.healun.2016.12.012.txt
ann_1471-2466-11-25.txt
ann_222.full.txt
ann_chest.10-0969.txt
ann_j.rmed.2017.05.008.txt
ann_j.healun.2006.11.009.txt
ann_chest.130.5.1471.txt
ann_03007995.2011.605440.txt
ann_j.healun.2008.04.009.txt
ann_j.1365-2141.2005.05625.x.txt
ann_chest.11-0404.txt
ann_s12872-017-0569-3.txt
ann_1410.full.txt
ann_j.jacc.2005.04.050.txt
ann_s40256-017-0262-z.txt
ann_s12872-016-0361-9.txt
ann_030079905x30680.txt
ann_j.jvca.2007.10.015.txt
ann_0003-4975_2895_2900408-d.txt
ann_bloodadvances.2019000883.txt
ann_j.1365-2362.2006.01688.x.txt
ann_09031936.04.00111904.txt
ann_09031936.00182909.txt
ann_j.healun.2013.06.008.txt
ann_72.full.txt
ann_1602493.full.txt
ann_NEJMoa1209655.txt
ann_CIRCRESAHA.114.305951.txt
ann_jcm-06-00043-v2.txt
ann_rccm.200605-694OC.txt
ann_hjh.0b013e3282f382ff.txt
ann_bcp.14039.txt
ann_Phosphodisesterase inhibitors in Pulmonary hypertension.txt
ann_2000513.full.txt
ann_j.ijcard.2010.10.051.txt
ann_s0140-6736_2895_2991504-4.txt
ann_691.full.txt
ann_j.

In [176]:
with open('/Users/aelitta/Documents/DMMatrix/PAH/CRE/spert/articles_inf_aa.json', 'w') as f:
    f.write(json.dumps(jsonlist_inf))

In [478]:
##for SEMVEAL

json_neg = []
cnt = 0
for p in pred:
    if len(p['entities'])>1:
        for e1 in p['entities']:
            for e2 in p['entities']:
                if e1!=e2:
                    json_neg.append({
                       'h': [[p['tokens'][e1['start']:e1['end']]], e1['type'].capitalize(), [str(y) for y in range(int(e1['start']),int(e1['end']))]],
                       't': [[p['tokens'][e2['start']:e2['end']]], e2['type'].capitalize(), [str(y) for y in range(int(e2['start']),int(e2['end']))]],
                        'tokens': p['tokens'],
                        'h_id': cnt+p['entities'].index(e1),
                        't_id': cnt+p['entities'].index(e2)
                    })
    cnt+=1

In [259]:
len(prob)

57733

In [154]:
s = ' '.join(pred[65]['tokens'][:-1])+'.'
sent[sent['sentence']==s]

,sentence_id,filename,sentence
61687,61688,s00246-007-9139-2.txt,Our data reveal that a single dose of oral sil...
